# Million Song Dataset Classification

Baseline tests concluded that the system with batch normalization achieved the best results. --CONTINUE--

Test to include in making the system better:

* Feedforward network with different input data
    * Lower number of segments (Reduce from 120 to 100, 80, 60, 40, 20) with **best baseline configuration**.
    * Lower number of segments with **computationally efficient best baseline configuration**.


* Convolutional network
    * Full data different configurations. Kernel Sizes, Maxpool Window size, network size (convolutional and feedfoward), batch normalization, dropout, learning rate.
    * Repeat the above tests accept optimizer and learning rate for lower number of data (Reduce from 120, to 100, 80, 60, 40, 20).
    
Test convolutional neural network with the kernel being used as scanning the time instances.
    
    
* Recurrent network








Things to do: 

Fix data provider to give:

* Variable number of snippets
* Less dimensions
* More dimensions

##### Tensorboard command: 
tensorboard --logdir=~/Programming/Python/mlpractical/notebooks/Coursework4/logs/tb

### Import Packages

In [1]:
# Import important packages
import time
import os
import tensorflow as tf
import numpy as np
import mlp.nn_utils as nnu
import mlp.audio_utils as auu
from mlp.data_providers import MSD10CustomGenreDataProvider, MSD25CustomGenreDataProvider
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Function to run graph
def run_session(graph, train_step, train_data, valid_data, inputs, targets, path, summary_op, num_epoch=50): 
    
    # Create writer files where the summaries are written
    train_bkwriter = tf.summary.FileWriter(os.path.join('Coursework4', 'logs', 'tb_bk', path, 'train'), graph=graph)
    valid_bkwriter = tf.summary.FileWriter(os.path.join('Coursework4', 'logs', 'tb_bk', path, 'valid'), graph=graph)
    valid_writer = tf.summary.FileWriter(os.path.join('Coursework4', 'logs', 'tb', path, 'valid'), graph=graph, flush_secs=10)
   
    
    # Intialize variables
    with graph.as_default():       
        init = tf.global_variables_initializer()
        
    # Define session and reset variable
    sess= tf.InteractiveSession(graph=graph)
    sess.run(init)
   
    for e in range(num_epoch):
        # Iterate through all the batches in train_data and compute
        for b, (input_batch, target_batch) in enumerate(train_data):
            _, summary = sess.run(
                [train_step, summary_op], 
                feed_dict={inputs: input_batch, targets: target_batch})
            train_bkwriter.add_summary(summary, e* train_data.num_batches + b)

        # Run validation set every 5 epochs of the training
        for r, (valid_input_batch, valid_target_batch) in enumerate(valid_data):
            valid_summary = sess.run(
                summary_op, feed_dict={inputs: valid_input_batch, targets: valid_target_batch})
            valid_bkwriter.add_summary(valid_summary, e* train_data.num_batches + r*
                                     (train_data.num_batches/valid_data.num_batches))
            valid_writer.add_summary(valid_summary, e* train_data.num_batches + r*
                                     (train_data.num_batches/valid_data.num_batches))
 
        # Run last epoch return mean accuracy and error of last epoch
        if e == num_epoch-1:
            last_epoch_errors=[]
            last_epoch_accuracies=[]
            for r, (valid_input_batch, valid_target_batch) in enumerate(valid_data):
                last_epoch_error, last_epoch_accuracy = sess.run(
                    [error, accuracy], feed_dict={inputs: valid_input_batch, targets: valid_target_batch})
                last_epoch_errors.append(last_epoch_error)
                last_epoch_accuracies.append(last_epoch_accuracy)
                
    return np.mean(last_epoch_errors), np.mean(last_epoch_accuracies)

### Import data

In [3]:
train_data_default = MSD10CustomGenreDataProvider('train', batch_size=50, dimension='default')
valid_data_default = MSD10CustomGenreDataProvider('valid', batch_size=50, dimension='default')
print(train_data_default.inputs.shape)

train_data_full = MSD10CustomGenreDataProvider('train', batch_size=50, dimension='full')
valid_data_full = MSD10CustomGenreDataProvider('valid', batch_size=50, dimension='full')
print(valid_data_full.inputs.shape)

train_data_fullNoChromaDeltas = MSD10CustomGenreDataProvider('train', batch_size=50, dimension='full-nochromd')
valid_data_fullNoChromaDeltas = MSD10CustomGenreDataProvider('valid', batch_size=50, dimension='full-nochromd')
print(train_data_fullNoChromaDeltas.inputs.shape)

train_data_timbreDeltas = MSD10CustomGenreDataProvider('train', batch_size=50, dimension='timbre-deltas')
valid_data_timbreDeltas = MSD10CustomGenreDataProvider('valid', batch_size=50, dimension='timbre-deltas')
print(train_data_timbreDeltas.inputs.shape)

train_data_timbre = MSD10CustomGenreDataProvider('train', batch_size=50, dimension='timbre')
valid_data_timbre = MSD10CustomGenreDataProvider('valid', batch_size=50, dimension='timbre')
print(train_data_timbre.inputs.shape)

print('Data Loaded!')

(40000, 120, 25, 1)
(10000, 120, 88, 1)
(40000, 120, 60, 1)
(40000, 120, 36, 1)
(40000, 120, 12, 1)
Data Loaded!


## Learning Rate Tests

In [9]:
epochs=20
train_data = train_data_fullNoChromaDeltas
valid_data = valid_data_fullNoChromaDeltas

nonLin=tf.nn.elu
freq_vector_size = train_data.inputs.shape[2]
print(freq_vector_size)

learning_rate_test_vals = [1e-4, 5e-4, 1e-3, 5e-3, 1e-2, 5e-2]

60


### Traning using Adam

#### Conv 1D

In [7]:
'''Conv1D'''
testname='conv1d_learningRate_Adam'

fcPs = [(1500,200),(200,train_data.num_classes)]
cnPs = [(1,10),(10,20),(20,30),(30,40)]
kPs = [(5,1),(5,1),(5,1),(5,1)]
mPPs = [(2,1),(2,1),(2,1),(3,1)]
results={}

for lr in learning_rate_test_vals:
    
    # Test paramter
    optimizer = tf.train.AdamOptimizer(learning_rate=lr)
    btcN = True
    
    #Define graph
    TGraph=tf.Graph()
    [layer_outs, error, accuracy, inputs, targets, sop, t_stp] = nnu.net_model(TGraph, train_data, 
                            convParams=cnPs, maxPoolParams=mPPs, kernelParams=kPs, cnNonLinearity=nonLin,
                            fcParams=fcPs, fcNonLinearity=nonLin, optimizer=optimizer, bnCn=btcN)

    
    # Perform test and save results in dictionary
    path = auu.create_fname(testname, [lr, 'batch']) # <<
    start_run = time.time()
    test_error, test_accuracy = run_session(TGraph, t_stp, train_data, valid_data, inputs, targets, path, sop, epochs)
    runtime = round(time.time() - start_run, 2)
    results[str(lr)] = [runtime, test_error, test_accuracy] #<<

    # Print results to console
    print("Runtime: {0}".format(runtime))
    print("Test Error: {0}\nTest Accuracy: {1}".format(test_error, test_accuracy))
    
# save CSV file and print final    
auu.dict2csv(results, '../notebooks/Coursework4/results/', testname)

Runtime: 1691.26
Test Error: 1.2894610166549683
Test Accuracy: 0.5478000044822693
Runtime: 1834.28
Test Error: 1.4058423042297363
Test Accuracy: 0.5074000358581543
Runtime: 2055.46
Test Error: 1.4850925207138062
Test Accuracy: 0.47669997811317444
Runtime: 1788.88
Test Error: 2.346517324447632
Test Accuracy: 0.10000000149011612
Runtime: 1854.12
Test Error: 2.425968647003174
Test Accuracy: 0.10000000149011612
Runtime: 1870.51
Test Error: 3.017825126647949
Test Accuracy: 0.10000000149011612


#### Con 2D Full Frequency

In [5]:
'''Conv2D Full Frequency'''
testname='conv2dFF_learningRate_Adam'

fcPs = [(1500,200),(200,train_data.num_classes)]
cnPs = [(1,10),(10,50),(50,100),(100,200)]
kPs = [(5,freq_vector_size),(5,1),(5,1),(5,1)]
mPPs = [(1,1),(2,1),(2,1),(29,1)]
results={}

padding=['VALID', 'SAME', 'SAME', 'SAME']

for lr in learning_rate_test_vals:
    
    # Test paramter
    optimizer = tf.train.AdamOptimizer(learning_rate=lr)
    btcN = True
    
    
    #Define graph
    TGraph=tf.Graph()
    [layer_outs, error, accuracy, inputs, targets, sop, t_stp] = nnu.net_model(TGraph, train_data, 
                            convParams=cnPs, maxPoolParams=mPPs, kernelParams=kPs, cnNonLinearity=nonLin,
                            fcParams=fcPs, fcNonLinearity=nonLin, optimizer=optimizer, bnCn=btcN, convPadding=padding)

    
    # Perform test and save results in dictionary
    path = auu.create_fname(testname, [lr, 'batch']) # <<
    start_run = time.time()
    test_error, test_accuracy = run_session(TGraph, t_stp, train_data, valid_data, inputs, targets, path, sop, epochs)
    runtime = round(time.time() - start_run, 2)
    results[str(lr)] = [runtime, test_error, test_accuracy] #<<

    # Print results to console
    print("Runtime: {0}".format(runtime))
    print("Test Error: {0}\nTest Accuracy: {1}".format(test_error, test_accuracy))
    
# save CSV file and print final    
auu.dict2csv(results, '../notebooks/Coursework4/results/', testname)

Runtime: 362.64
Test Error: 1.629669189453125
Test Accuracy: 0.41929998993873596
Runtime: 348.82
Test Error: 1.426578402519226
Test Accuracy: 0.48879995942115784
Runtime: 357.35
Test Error: 1.3235278129577637
Test Accuracy: 0.5428999662399292
Runtime: 354.94
Test Error: 1.6616209745407104
Test Accuracy: 0.5085999965667725
Runtime: 359.87
Test Error: 1.4332122802734375
Test Accuracy: 0.5312999486923218
Runtime: 350.71
Test Error: 3.235905408859253
Test Accuracy: 0.10000000149011612


In [11]:
'''Conv2D Full Frequency'''
testname='conv2dFF_learningRate_Adam_nobatch'

fcPs = [(1500,200),(200,train_data.num_classes)]
cnPs = [(1,10),(10,50),(50,100),(100,200)]
kPs = [(5,freq_vector_size),(5,1),(5,1),(5,1)]
mPPs = [(1,1),(2,1),(2,1),(29,1)]
results={}

padding=['VALID', 'SAME', 'SAME', 'SAME']

for lr in [1e-6, 1e-5, 1e-4]:
    
    # Test paramter
    optimizer = tf.train.AdamOptimizer(learning_rate=lr)
    btcN = False
    
    
    #Define graph
    TGraph=tf.Graph()
    [layer_outs, error, accuracy, inputs, targets, sop, t_stp] = nnu.net_model(TGraph, train_data, 
                            convParams=cnPs, maxPoolParams=mPPs, kernelParams=kPs, cnNonLinearity=nonLin,
                            fcParams=fcPs, fcNonLinearity=nonLin, optimizer=optimizer, bnCn=btcN, convPadding=padding)

    
    # Perform test and save results in dictionary
    path = auu.create_fname(testname, [lr, 'batch']) # <<
    start_run = time.time()
    test_error, test_accuracy = run_session(TGraph, t_stp, train_data, valid_data, inputs, targets, path, sop, epochs)
    runtime = round(time.time() - start_run, 2)
    results[str(lr)] = [runtime, test_error, test_accuracy] #<<

    # Print results to console
    print("Runtime: {0}".format(runtime))
    print("Test Error: {0}\nTest Accuracy: {1}".format(test_error, test_accuracy))
    
# save CSV file and print final    
auu.dict2csv(results, '../notebooks/Coursework4/results/', testname)

Runtime: 282.29
Test Error: 13776.8515625
Test Accuracy: 0.10989999771118164
Runtime: 290.53
Test Error: 5677.1533203125
Test Accuracy: 0.15690000355243683
Runtime: 276.34
Test Error: 250.16209411621094
Test Accuracy: 0.20250000059604645


In [15]:
'''Conv2D Full Frequency'''
testname='conv2dFF_learningRate_AdaDelta'

fcPs = [(1500,200),(200,train_data.num_classes)]
cnPs = [(1,10),(10,50),(50,100),(100,200)]
kPs = [(5,freq_vector_size),(5,1),(5,1),(5,1)]
mPPs = [(1,1),(2,1),(2,1),(29,1)]
results={}

padding=['VALID', 'SAME', 'SAME', 'SAME']

for lr in [1e-2, 1e-1]:
    
    # Test paramter
    optimizer = tf.train.AdadeltaOptimizer(learning_rate=lr)
    btcN = True
    
    
    #Define graph
    TGraph=tf.Graph()
    [layer_outs, error, accuracy, inputs, targets, sop, t_stp] = nnu.net_model(TGraph, train_data, 
                            convParams=cnPs, maxPoolParams=mPPs, kernelParams=kPs, cnNonLinearity=nonLin,
                            fcParams=fcPs, fcNonLinearity=nonLin, optimizer=optimizer, bnCn=btcN, convPadding=padding)

    
    # Perform test and save results in dictionary
    path = auu.create_fname(testname, [lr, 'batch']) # <<
    start_run = time.time()
    test_error, test_accuracy = run_session(TGraph, t_stp, train_data, valid_data, inputs, targets, path, sop, epochs)
    runtime = round(time.time() - start_run, 2)
    results[str(lr)] = [runtime, test_error, test_accuracy] #<<

    # Print results to console
    print("Runtime: {0}".format(runtime))
    print("Test Error: {0}\nTest Accuracy: {1}".format(test_error, test_accuracy))
    
# save CSV file and print final    
auu.dict2csv(results, '../notebooks/Coursework4/results/', testname)

Runtime: 358.11
Test Error: 1.660836935043335
Test Accuracy: 0.398499995470047
Runtime: 342.11
Test Error: 2.3028151988983154
Test Accuracy: 0.10000000149011612


#### Conv 2D Floating Kernel

In [10]:
'''Conv2D'''
testname='conv2d_learningRate_Adam'

fcPs = [(1500,200),(200,train_data.num_classes)]
cnPs = [(1,10),(10,30),(30,50),(50,70)]
kPs = [(3,3),(3,3),(3,3),(3,3)]
mPPs = [(2,2),(2,1),(2,2),(3,3)]
results={}

for lr in learning_rate_test_vals:
    
    # Test paramter
    optimizer = tf.train.AdamOptimizer(learning_rate=lr)
    btcN = True
    
    #Define graph
    TGraph=tf.Graph()
    [layer_outs, error, accuracy, inputs, targets, sop, t_stp] = nnu.net_model(TGraph, train_data, 
                            convParams=cnPs, maxPoolParams=mPPs, kernelParams=kPs, cnNonLinearity=nonLin,
                            fcParams=fcPs, fcNonLinearity=nonLin, optimizer=optimizer, bnCn=btcN)

    
    # Perform test and save results in dictionary
    path = auu.create_fname(testname, [lr, 'batch']) # <<
    start_run = time.time()
    test_error, test_accuracy = run_session(TGraph, t_stp, train_data, valid_data, inputs, targets, path, sop, epochs)
    runtime = round(time.time() - start_run, 2)
    results[str(lr)] = [runtime, test_error, test_accuracy] #<<

    # Print results to console
    print("Runtime: {0}".format(runtime))
    print("Test Error: {0}\nTest Accuracy: {1}".format(test_error, test_accuracy))
    
# save CSV file and print final    
auu.dict2csv(results, '../notebooks/Coursework4/results/', testname)

Runtime: 1549.1
Test Error: 1.5013319253921509
Test Accuracy: 0.46389999985694885
Runtime: 1766.59
Test Error: 1.3122329711914062
Test Accuracy: 0.5374000072479248
Runtime: 1729.27
Test Error: 1.3069031238555908
Test Accuracy: 0.5442999601364136
Runtime: 1666.63
Test Error: 1.4124581813812256
Test Accuracy: 0.5465999841690063
Runtime: 1526.94
Test Error: 2.4137990474700928
Test Accuracy: 0.10000000149011612
Runtime: 1370.61
Test Error: 3.08671236038208
Test Accuracy: 0.10000000149011612


### Traning using SGD

#### Conv 2D Full Frequency

In [6]:
'''Conv2D Full Frequency'''
testname='conv2dFF_learningRate_SGD'

fcPs = [(1500,200),(200,train_data.num_classes)]
cnPs = [(1,10),(10,50),(50,100),(100,200)]
kPs = [(5,freq_vector_size),(5,1),(5,1),(5,1)]
mPPs = [(1,1),(2,1),(2,1),(29,1)]
results={}

padding=['VALID', 'SAME', 'SAME', 'SAME']

for lr in learning_rate_test_vals:
    
    # Test paramter
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
    btcN = True
    
    
    #Define graph
    TGraph=tf.Graph()
    [layer_outs, error, accuracy, inputs, targets, sop, t_stp] = nnu.net_model(TGraph, train_data, 
                            convParams=cnPs, maxPoolParams=mPPs, kernelParams=kPs, cnNonLinearity=nonLin,
                            fcParams=fcPs, fcNonLinearity=nonLin, optimizer=optimizer, bnCn=btcN, convPadding=padding)

    
    # Perform test and save results in dictionary
    path = auu.create_fname(testname, [lr, 'batch']) # <<
    start_run = time.time()
    test_error, test_accuracy = run_session(TGraph, t_stp, train_data, valid_data, inputs, targets, path, sop, epochs)
    runtime = round(time.time() - start_run, 2)
    results[str(lr)] = [runtime, test_error, test_accuracy] #<<

    # Print results to console
    print("Runtime: {0}".format(runtime))
    print("Test Error: {0}\nTest Accuracy: {1}".format(test_error, test_accuracy))
    
# save CSV file and print final    
auu.dict2csv(results, '../notebooks/Coursework4/results/', testname)

Runtime: 338.66
Test Error: 2.302905797958374
Test Accuracy: 0.09999999403953552
Runtime: 340.98
Test Error: 2.3043487071990967
Test Accuracy: 0.10000000149011612
Runtime: 389.76
Test Error: 2.3059492111206055
Test Accuracy: 0.10000000149011612
Runtime: 354.9
Test Error: 1.783705472946167
Test Accuracy: 0.3513999879360199
Runtime: 347.04
Test Error: 1.5800763368606567
Test Accuracy: 0.4358000159263611
Runtime: 348.76
Test Error: 1.5035948753356934
Test Accuracy: 0.4619000256061554


## Mode Tests

In [13]:
epochs=20

nonLin=tf.nn.elu

mode_test_vals = ['default', 'timbre', 'full', 'timbre-deltas']

#### Conv 1D

In [16]:
'''Conv1D'''
testname='conv1d_data'
lr = 1e-4

fcPs = [(1500,200),(200,train_data.num_classes)]
cnPs = [(1,10),(10,20),(20,30),(30,40)]
kPs = [(5,1),(5,1),(5,1),(5,1)]
mPPs = [(2,1),(2,1),(2,1),(3,1)]
results={}

for mode in mode_test_vals:
    
    if mode=='default':
        train_data = train_data_default
        valid_data = valid_data_default
    elif mode =='timbre':
        train_data = train_data_timbre
        valid_data = valid_data_timbre
    elif mode=='full':
        train_data = train_data_full
        valid_data = valid_data_full
    else:
        train_data = train_data_timbreDeltas
        valid_data = valid_data_timbreDeltas
    
    # Test paramter
    optimizer = tf.train.AdamOptimizer(learning_rate=lr)
    btcN = True
    
    #Define graph
    TGraph=tf.Graph()
    [layer_outs, error, accuracy, inputs, targets, sop, t_stp] = nnu.net_model(TGraph, train_data, 
                            convParams=cnPs, maxPoolParams=mPPs, kernelParams=kPs, cnNonLinearity=nonLin,
                            fcParams=fcPs, fcNonLinearity=nonLin, optimizer=optimizer, bnCn=btcN)

    
    # Perform test and save results in dictionary
    path = auu.create_fname(testname, [mode]) # <<
    start_run = time.time()
    test_error, test_accuracy = run_session(TGraph, t_stp, train_data, valid_data, inputs, targets, path, sop, epochs)
    runtime = round(time.time() - start_run, 2)
    results[mode] = [runtime, test_error, test_accuracy] #<<

    # Print results to console
    print("Runtime: {0}".format(runtime))
    print("Test Error: {0}\nTest Accuracy: {1}".format(test_error, test_accuracy))
    
# save CSV file and print final    
auu.dict2csv(results, '../notebooks/Coursework4/results/', testname)

Runtime: 769.31
Test Error: 1.3177273273468018
Test Accuracy: 0.540399968624115
Runtime: 438.63
Test Error: 1.3943997621536255
Test Accuracy: 0.5053999423980713
Runtime: 2543.12
Test Error: 1.265673041343689
Test Accuracy: 0.5570999979972839
Runtime: 1067.52
Test Error: 1.328257441520691
Test Accuracy: 0.5374000072479248


#### Con 2D Full Frequency

In [17]:
'''Conv2D Full Frequency'''
testname='conv2dFF_data'
lr = 1e-3

fcPs = [(1500,200),(200,train_data.num_classes)]
cnPs = [(1,10),(10,50),(50,100),(100,200)]
mPPs = [(1,1),(2,1),(2,1),(29,1)]
results={}

padding=['VALID', 'SAME', 'SAME', 'SAME']

for mode in mode_test_vals:
    
    if mode=='default':
        train_data = train_data_default
        valid_data = valid_data_default
        kPs = [(5,25),(5,1),(5,1),(5,1)]
    elif mode =='timbre':
        train_data = train_data_timbre
        valid_data = valid_data_timbre
        kPs = [(5,12),(5,1),(5,1),(5,1)]
    elif mode=='full':
        train_data = train_data_full
        valid_data = valid_data_full
        kPs = [(5,88),(5,1),(5,1),(5,1)]
    else:
        train_data = train_data_timbreDeltas
        valid_data = valid_data_timbreDeltas
        kPs = [(5,36),(5,1),(5,1),(5,1)]
    
    # Test paramter
    optimizer = tf.train.AdamOptimizer(learning_rate=lr)
    btcN = True
    
    
    #Define graph
    TGraph=tf.Graph()
    [layer_outs, error, accuracy, inputs, targets, sop, t_stp] = nnu.net_model(TGraph, train_data, 
                            convParams=cnPs, maxPoolParams=mPPs, kernelParams=kPs, cnNonLinearity=nonLin,
                            fcParams=fcPs, fcNonLinearity=nonLin, optimizer=optimizer, bnCn=btcN, convPadding=padding)

    
    # Perform test and save results in dictionary
    path = auu.create_fname(testname, [mode]) # <<
    start_run = time.time()
    test_error, test_accuracy = run_session(TGraph, t_stp, train_data, valid_data, inputs, targets, path, sop, epochs)
    runtime = round(time.time() - start_run, 2)
    results[mode] = [runtime, test_error, test_accuracy] #<<

    # Print results to console
    print("Runtime: {0}".format(runtime))
    print("Test Error: {0}\nTest Accuracy: {1}".format(test_error, test_accuracy))
    
# save CSV file and print final    
auu.dict2csv(results, '../notebooks/Coursework4/results/', testname)

Runtime: 302.77
Test Error: 1.3035364151000977
Test Accuracy: 0.5388999581336975
Runtime: 281.83
Test Error: 1.3423304557800293
Test Accuracy: 0.5267999768257141
Runtime: 387.9
Test Error: 1.3324700593948364
Test Accuracy: 0.5311999320983887
Runtime: 315.24
Test Error: 1.321275234222412
Test Accuracy: 0.533799946308136


#### Conv 2D Floating Kernel

In [13]:
'''Conv2D'''
testname='conv2d_data'
lr = 5e-3

fcPs = [(1500,200),(200,train_data.num_classes)]
cnPs = [(1,10),(10,30),(30,50),(50,70)]
kPs = [(3,3),(3,3),(3,3),(3,3)]
results={}

for mode in mode_test_vals:
    
    if mode=='default':
        train_data = train_data_default
        valid_data = valid_data_default
        mPPs = [(2,1),(2,1),(2,1),(3,5)]
    elif mode =='timbre':
        train_data = train_data_timbre
        valid_data = valid_data_timbre
        mPPs = [(2,1),(2,1),(2,1),(3,3)]
    elif mode=='full':
        train_data = train_data_full
        valid_data = valid_data_full
        mPPs = [(2,2),(2,2),(2,2),(3,1)]
    else:
        train_data = train_data_timbreDeltas
        valid_data = valid_data_timbreDeltas
        mPPs = [(2,1),(2,2),(2,2),(3,1)]
    
    # Test paramter
    optimizer = tf.train.AdamOptimizer(learning_rate=lr)
    btcN = True
    
    #Define graph
    TGraph=tf.Graph()
    [layer_outs, error, accuracy, inputs, targets, sop, t_stp] = nnu.net_model(TGraph, train_data, 
                            convParams=cnPs, maxPoolParams=mPPs, kernelParams=kPs, cnNonLinearity=nonLin,
                            fcParams=fcPs, fcNonLinearity=nonLin, optimizer=optimizer, bnCn=btcN)

    
    # Perform test and save results in dictionary
    path = auu.create_fname(testname, [mode]) # <<
    start_run = time.time()
    test_error, test_accuracy = run_session(TGraph, t_stp, train_data, valid_data, inputs, targets, path, sop, epochs)
    runtime = round(time.time() - start_run, 2)
    results[mode] = [runtime, test_error, test_accuracy] #<<

    # Print results to console
    print("Runtime: {0}".format(runtime))
    print("Test Error: {0}\nTest Accuracy: {1}".format(test_error, test_accuracy))
    
# save CSV file and print final    
auu.dict2csv(results, '../notebooks/Coursework4/results/', testname)

Runtime: 1176.37
Test Error: 1.3052700757980347
Test Accuracy: 0.5609999299049377
Runtime: 753.0
Test Error: 1.6124228239059448
Test Accuracy: 0.5166999697685242
Runtime: 1949.61
Test Error: 2.347338914871216
Test Accuracy: 0.10000000149011612
Runtime: 1204.65
Test Error: 2.1889312267303467
Test Accuracy: 0.4943999946117401


## Kernel Size Test

In [4]:
epochs=20

nonLin=tf.nn.elu

### Traning using Adam

#### Conv 1D

In [5]:
'''Conv1D'''
testname='conv1d_kerSize_2'
lr = 1e-4

train_data = train_data_full
valid_data = valid_data_full

fcPs = [(1500,200),(200,train_data.num_classes)]
cnPs = [(1,10),(10,20),(20,30),(30,40)]
mPPs = [(2,1),(2,1),(2,1),(3,1)]
results={}

for ks in [15, 33, 66, 88, 100]:
    
    kPs = [(ks,1),(ks,1),(ks,1),(ks,1)]
    
    # Test paramter
    optimizer = tf.train.AdamOptimizer(learning_rate=lr)
    btcN = True
    
    #Define graph
    TGraph=tf.Graph()
    [layer_outs, error, accuracy, inputs, targets, sop, t_stp] = nnu.net_model(TGraph, train_data, 
                            convParams=cnPs, maxPoolParams=mPPs, kernelParams=kPs, cnNonLinearity=nonLin,
                            fcParams=fcPs, fcNonLinearity=nonLin, optimizer=optimizer, bnCn=btcN)

    
    # Perform test and save results in dictionary
    path = auu.create_fname(testname, [ks]) # <<
    start_run = time.time()
    test_error, test_accuracy = run_session(TGraph, t_stp, train_data, valid_data, inputs, targets, path, sop, epochs)
    runtime = round(time.time() - start_run, 2)
    results[ks] = [runtime, test_error, test_accuracy] #<<

    # Print results to console
    print("Runtime: {0}".format(runtime))
    print("Test Error: {0}\nTest Accuracy: {1}".format(test_error, test_accuracy))
    
# save CSV file and print final    
auu.dict2csv(results, '../notebooks/Coursework4/results/', testname)

Runtime: 2752.73
Test Error: 1.2723042964935303
Test Accuracy: 0.5632999539375305
Runtime: 3615.86
Test Error: 1.2687182426452637
Test Accuracy: 0.5539999604225159
Runtime: 5245.06
Test Error: 1.2853511571884155
Test Accuracy: 0.5538999438285828
Runtime: 6175.62
Test Error: 1.351770043373108
Test Accuracy: 0.5282999277114868


KeyboardInterrupt: 

#### Con 2D Full Frequency

In [ ]:
'''Conv2D Full Frequency'''
testname='conv2dFF_kerSize_2'
lr = 1e-3

train_data = train_data_fullNoChromaDeltas
valid_data = valid_data_fullNoChromaDeltas
freq_vector_size = train_data.inputs.shape[2]

fcPs = [(1500,200),(200,train_data.num_classes)]
cnPs = [(1,10),(10,50),(50,100),(100,200)]
results={}

padding=['VALID', 'SAME', 'SAME', 'SAME']

for ks in [15, 33, 66, 88, 100]:
    
    kPs = [(ks,freq_vector_size),(ks,1),(ks,1),(ks,1)]
    
    if ks==3:
        mPPs = [(1,1),(2,1),(1,1),(59,1)]
    elif ks==7:
        mPPs = [(1,1),(2,1),(3,1),(19,1)]
    elif ks==9:
        mPPs = [(2,1),(2,1),(2,1),(14,1)]
    
    # Test paramter
    optimizer = tf.train.AdamOptimizer(learning_rate=lr)
    btcN = True
    
    
    #Define graph
    TGraph=tf.Graph()
    [layer_outs, error, accuracy, inputs, targets, sop, t_stp] = nnu.net_model(TGraph, train_data, 
                            convParams=cnPs, maxPoolParams=mPPs, kernelParams=kPs, cnNonLinearity=nonLin,
                            fcParams=fcPs, fcNonLinearity=nonLin, optimizer=optimizer, bnCn=btcN, convPadding=padding)

    
    # Perform test and save results in dictionary
    path = auu.create_fname(testname, [ks]) # <<
    start_run = time.time()
    test_error, test_accuracy = run_session(TGraph, t_stp, train_data, valid_data, inputs, targets, path, sop, epochs)
    runtime = round(time.time() - start_run, 2)
    results[ks] = [runtime, test_error, test_accuracy] #<<

    # Print results to console
    print("Runtime: {0}".format(runtime))
    print("Test Error: {0}\nTest Accuracy: {1}".format(test_error, test_accuracy))
    
# save CSV file and print final    
auu.dict2csv(results, '../notebooks/Coursework4/results/', testname)

#### Conv 2D Floating Kernel

In [ ]:
'''Conv2D'''
testname='conv2d_kerSize'
lr = 5e-3

train_data = train_data_fullNoChromaDeltas
valid_data = valid_data_fullNoChromaDeltas

fcPs = [(1500,200),(200,train_data.num_classes)]
cnPs = [(1,10),(10,30),(30,50),(50,70)]
mPPs = [(2,2),(2,1),(2,2),(3,3)]
results={}

for ks in [15, 33, 66, 88, 100]:
    
    kPs = [(ks,ks),(ks,ks),(ks,ks),(ks,ks)]
    
    # Test paramter
    optimizer = tf.train.AdamOptimizer(learning_rate=lr)
    btcN = True
    
    #Define graph
    TGraph=tf.Graph()
    [layer_outs, error, accuracy, inputs, targets, sop, t_stp] = nnu.net_model(TGraph, train_data, 
                            convParams=cnPs, maxPoolParams=mPPs, kernelParams=kPs, cnNonLinearity=nonLin,
                            fcParams=fcPs, fcNonLinearity=nonLin, optimizer=optimizer, bnCn=btcN)

    
    # Perform test and save results in dictionary
    path = auu.create_fname(testname, [ks]) # <<
    start_run = time.time()
    test_error, test_accuracy = run_session(TGraph, t_stp, train_data, valid_data, inputs, targets, path, sop, epochs)
    runtime = round(time.time() - start_run, 2)
    results[ks] = [runtime, test_error, test_accuracy] #<<

    # Print results to console
    print("Runtime: {0}".format(runtime))
    print("Test Error: {0}\nTest Accuracy: {1}".format(test_error, test_accuracy))
    
# save CSV file and print final    
auu.dict2csv(results, '../notebooks/Coursework4/results/', testname)

## Network Size Test

In [4]:
epochs=20

nonLin=tf.nn.elu

size_tests =[1,2]

In [5]:
'''Conv1D'''
testname='conv1d_NetSize'
lr = 1e-4

train_data = train_data_full
valid_data = valid_data_full
results={}

size_tests = [1,2,3,4]
fcPs = [(1500,200),(200,train_data.num_classes)]
mPPs = [(2,1),(2,1),(2,1),(3,1)]
kPs = [(3,1),(3,1),(3,1),(3,1)]

for st in size_tests:
    
    if st==1: 
        cnPs = [(1,5),(5,10),(10,20),(20,30)]
    elif st==2:
        cnPs = [(1,2),(2,4),(4,8),(8,16)]
    elif st==3:
        cnPs = [(1,2),(2,3),(3,6),(6,9)]
    else:
        cnPs = [(1,2),(2,3),(3,4),(4,5)]

    
    # Test paramter
    optimizer = tf.train.AdamOptimizer(learning_rate=lr)
    btcN = True
    
    #Define graph
    TGraph=tf.Graph()
    [layer_outs, error, accuracy, inputs, targets, sop, t_stp] = nnu.net_model(TGraph, train_data, 
                            convParams=cnPs, maxPoolParams=mPPs, kernelParams=kPs, cnNonLinearity=nonLin,
                            fcParams=fcPs, fcNonLinearity=nonLin, optimizer=optimizer, bnCn=btcN)

    
    # Perform test and save results in dictionary
    path = auu.create_fname(testname, [st]) # <<
    start_run = time.time()
    test_error, test_accuracy = run_session(TGraph, t_stp, train_data, valid_data, inputs, targets, path, sop, epochs)
    runtime = round(time.time() - start_run, 2)
    results[st] = [runtime, test_error, test_accuracy] #<<

    # Print results to console
    print("Runtime: {0}".format(runtime))
    print("Test Error: {0}\nTest Accuracy: {1}".format(test_error, test_accuracy))
    
# save CSV file and print final    
auu.dict2csv(results, '../notebooks/Coursework4/results/', testname)

Runtime: 1815.94
Test Error: 1.2429964542388916
Test Accuracy: 0.5649999380111694
Runtime: 1561.37
Test Error: 1.262000560760498
Test Accuracy: 0.5606999397277832
Runtime: 1590.39
Test Error: 1.2952401638031006
Test Accuracy: 0.5453000068664551
Runtime: 1470.65
Test Error: 1.3059223890304565
Test Accuracy: 0.5467000007629395


#### Con 2D Full Frequency

In [29]:
'''Conv2D Full Frequency'''
testname='conv2dFF_NetSize'
lr = 1e-3

train_data = train_data_fullNoChromaDeltas
valid_data = valid_data_fullNoChromaDeltas
freq_vector_size = train_data.inputs.shape[2]
results={}

padding=['VALID', 'SAME', 'SAME', 'SAME']

fcPs = [(1500,200),(200,train_data.num_classes)]
kPs = [(3,freq_vector_size),(3,1),(3,1),(3,1)]
mPPs = [(1,1),(2,1),(2,1),(29,1)]
    
for st in size_tests:
    
    if st==1: 
        cnPs = [(1,20),(20,100),(100,200),(200,400)]
        
    else:
        cnPs = [(1,5),(5,10),(10,20),(20,200)]
    
    # Test paramter
    optimizer = tf.train.AdamOptimizer(learning_rate=lr)
    btcN = True
    
    
    #Define graph
    TGraph=tf.Graph()
    [layer_outs, error, accuracy, inputs, targets, sop, t_stp] = nnu.net_model(TGraph, train_data, 
                            convParams=cnPs, maxPoolParams=mPPs, kernelParams=kPs, cnNonLinearity=nonLin,
                            fcParams=fcPs, fcNonLinearity=nonLin, optimizer=optimizer, bnCn=btcN, convPadding=padding)

    
    # Perform test and save results in dictionary
    path = auu.create_fname(testname, [st]) # <<
    start_run = time.time()
    test_error, test_accuracy = run_session(TGraph, t_stp, train_data, valid_data, inputs, targets, path, sop, epochs)
    runtime = round(time.time() - start_run, 2)
    results[st] = [runtime, test_error, test_accuracy] #<<

    # Print results to console
    print("Runtime: {0}".format(runtime))
    print("Test Error: {0}\nTest Accuracy: {1}".format(test_error, test_accuracy))
    
# save CSV file and print final    
auu.dict2csv(results, '../notebooks/Coursework4/results/', testname)

Runtime: 586.7
Test Error: 1.3293771743774414
Test Accuracy: 0.5414999723434448
Runtime: 283.62
Test Error: 1.4303994178771973
Test Accuracy: 0.4883999526500702


#### Conv 2D Floating Kernel

In [30]:
'''Conv2D'''
testname='conv2d_NetSize'
lr = 5e-3

train_data = train_data_fullNoChromaDeltas
valid_data = valid_data_fullNoChromaDeltas

fcPs = [(1500,200),(200,train_data.num_classes)]
mPPs = [(2,2),(2,1),(2,2),(3,3)]
kPs = [(3,3),(3,3),(3,3),(3,3)]
results={}
    
for st in size_tests:
    
    if st==1: 
        cnPs = [(1,20),(20,50),(50,100),(100,200)]
    else:
        cnPs = [(1,5),(5,10),(10,15),(15,20)]
    
    
    # Test paramter
    optimizer = tf.train.AdamOptimizer(learning_rate=lr)
    btcN = True
    
    #Define graph
    TGraph=tf.Graph()
    [layer_outs, error, accuracy, inputs, targets, sop, t_stp] = nnu.net_model(TGraph, train_data, 
                            convParams=cnPs, maxPoolParams=mPPs, kernelParams=kPs, cnNonLinearity=nonLin,
                            fcParams=fcPs, fcNonLinearity=nonLin, optimizer=optimizer, bnCn=btcN)

    
    # Perform test and save results in dictionary
    path = auu.create_fname(testname, [st]) # <<
    start_run = time.time()
    test_error, test_accuracy = run_session(TGraph, t_stp, train_data, valid_data, inputs, targets, path, sop, epochs)
    runtime = round(time.time() - start_run, 2)
    results[st] = [runtime, test_error, test_accuracy] #<<

    # Print results to console
    print("Runtime: {0}".format(runtime))
    print("Test Error: {0}\nTest Accuracy: {1}".format(test_error, test_accuracy))
    
# save CSV file and print final    
auu.dict2csv(results, '../notebooks/Coursework4/results/', testname)

Runtime: 2641.67
Test Error: 2.3880724906921387
Test Accuracy: 0.10000000149011612
Runtime: 901.95
Test Error: 1.5716500282287598
Test Accuracy: 0.5067999958992004


## ------------------------------------------------------------------------------------------------------------------------------

## 25-Genre

### Import data

In [5]:
train_data_default25 = MSD25CustomGenreDataProvider('train', batch_size=50, dimension='default')
valid_data_default25 = MSD25CustomGenreDataProvider('valid', batch_size=50, dimension='default')
print(train_data_default25.inputs.shape)

train_data_full25 = MSD25CustomGenreDataProvider('train', batch_size=50, dimension='full')
valid_data_full25 = MSD25CustomGenreDataProvider('valid', batch_size=50, dimension='full')
print(valid_data_full25.inputs.shape)

train_data_fullNoChromaDeltas25 = MSD25CustomGenreDataProvider('train', batch_size=50, dimension='full-nochromd')
valid_data_fullNoChromaDeltas25 = MSD25CustomGenreDataProvider('valid', batch_size=50, dimension='full-nochromd')
print(train_data_fullNoChromaDeltas25.inputs.shape)

train_data_timbreDeltas25 = MSD25CustomGenreDataProvider('train', batch_size=50, dimension='timbre-deltas')
valid_data_timbreDeltas25 = MSD25CustomGenreDataProvider('valid', batch_size=50, dimension='timbre-deltas')
print(train_data_timbreDeltas25.inputs.shape)

train_data_timbre25 = MSD25CustomGenreDataProvider('train', batch_size=50, dimension='timbre')
valid_data_timbre25 = MSD25CustomGenreDataProvider('valid', batch_size=50, dimension='timbre')
print(train_data_timbre25.inputs.shape)

print('Data Loaded!')

(40000, 120, 25, 1)
(10000, 120, 88, 1)
(40000, 120, 60, 1)
(40000, 120, 36, 1)
(40000, 120, 12, 1)
Data Loaded!


## Learning Rate Tests

In [7]:
epochs=20
train_data = train_data_fullNoChromaDeltas25
valid_data = valid_data_fullNoChromaDeltas25

nonLin=tf.nn.elu
freq_vector_size = train_data.inputs.shape[2]
print(freq_vector_size)

learning_rate_test_vals = [1e-4, 5e-4, 1e-3, 5e-3, 1e-2, 5e-2]

60


### Traning using Adam

#### Conv 1D

In [5]:
'''Conv1D'''
testname='conv1d_learningRate_Adam_25'

fcPs = [(1500,200),(200,train_data.num_classes)]
cnPs = [(1,10),(10,20),(20,30),(30,40)]
kPs = [(5,1),(5,1),(5,1),(5,1)]
mPPs = [(2,1),(2,1),(2,1),(3,1)]
results={}

for lr in learning_rate_test_vals:
    
    # Test paramter
    optimizer = tf.train.AdamOptimizer(learning_rate=lr)
    btcN = True
    
    #Define graph
    TGraph=tf.Graph()
    [layer_outs, error, accuracy, inputs, targets, sop, t_stp] = nnu.net_model(TGraph, train_data, 
                            convParams=cnPs, maxPoolParams=mPPs, kernelParams=kPs, cnNonLinearity=nonLin,
                            fcParams=fcPs, fcNonLinearity=nonLin, optimizer=optimizer, bnCn=btcN)

    
    # Perform test and save results in dictionary
    path = auu.create_fname(testname, [lr, 'batch']) # <<
    start_run = time.time()
    test_error, test_accuracy = run_session(TGraph, t_stp, train_data, valid_data, inputs, targets, path, sop, epochs)
    runtime = round(time.time() - start_run, 2)
    results[str(lr)] = [runtime, test_error, test_accuracy] #<<

    # Print results to console
    print("Runtime: {0}".format(runtime))
    print("Test Error: {0}\nTest Accuracy: {1}".format(test_error, test_accuracy))
    
# save CSV file and print final    
auu.dict2csv(results, '../notebooks/Coursework4/results/', testname)

Runtime: 1690.69
Test Error: 2.5420641899108887
Test Accuracy: 0.23680000007152557
Runtime: 1723.81
Test Error: 2.86385440826416
Test Accuracy: 0.10700000822544098
Runtime: 1730.56
Test Error: 2.897284507751465
Test Accuracy: 0.10179999470710754
Runtime: 1550.25
Test Error: 3.3122968673706055
Test Accuracy: 0.03999999910593033
Runtime: 1551.59
Test Error: 3.543579339981079
Test Accuracy: 0.03999999910593033
Runtime: 1562.55
Test Error: 4.473163604736328
Test Accuracy: 0.03999999910593033


#### Conv 2D Full Frequency

In [8]:
'''Conv2D Full Frequency'''
testname='conv2dFF_learningRate_Adam_25'

fcPs = [(1500,200),(200,train_data.num_classes)]
cnPs = [(1,10),(10,50),(50,100),(100,200)]
kPs = [(5,freq_vector_size),(5,1),(5,1),(5,1)]
mPPs = [(1,1),(2,1),(2,1),(29,1)]
results={}

padding=['VALID', 'SAME', 'SAME', 'SAME']

for lr in learning_rate_test_vals:
    
    # Test paramter
    optimizer = tf.train.AdamOptimizer(learning_rate=lr)
    btcN = True
    
    
    #Define graph
    TGraph=tf.Graph()
    [layer_outs, error, accuracy, inputs, targets, sop, t_stp] = nnu.net_model(TGraph, train_data, 
                            convParams=cnPs, maxPoolParams=mPPs, kernelParams=kPs, cnNonLinearity=nonLin,
                            fcParams=fcPs, fcNonLinearity=nonLin, optimizer=optimizer, bnCn=btcN, convPadding=padding)

    
    # Perform test and save results in dictionary
    path = auu.create_fname(testname, [lr, 'batch']) # <<
    start_run = time.time()
    test_error, test_accuracy = run_session(TGraph, t_stp, train_data, valid_data, inputs, targets, path, sop, epochs)
    runtime = round(time.time() - start_run, 2)
    results[str(lr)] = [runtime, test_error, test_accuracy] #<<

    # Print results to console
    print("Runtime: {0}".format(runtime))
    print("Test Error: {0}\nTest Accuracy: {1}".format(test_error, test_accuracy))
    
# save CSV file and print final    
auu.dict2csv(results, '../notebooks/Coursework4/results/', testname)

Runtime: 348.31
Test Error: 2.7217133045196533
Test Accuracy: 0.1779000163078308
Runtime: 355.27
Test Error: 2.534174919128418
Test Accuracy: 0.23319999873638153
Runtime: 359.66
Test Error: 2.5458245277404785
Test Accuracy: 0.23099999129772186
Runtime: 356.04
Test Error: 2.5600829124450684
Test Accuracy: 0.2363000065088272
Runtime: 368.33
Test Error: 2.5466785430908203
Test Accuracy: 0.23170000314712524
Runtime: 382.41
Test Error: 4.5097336769104
Test Accuracy: 0.03999999910593033


#### Conv 2D Floating Kernel

In [5]:
'''Conv2D'''
testname='conv2d_learningRate_Adam_25'

fcPs = [(1500,200),(200,train_data.num_classes)]
cnPs = [(1,10),(10,30),(30,50),(50,70)]
kPs = [(3,3),(3,3),(3,3),(3,3)]
mPPs = [(2,2),(2,1),(2,2),(3,3)]
results={}

for lr in learning_rate_test_vals:
    
    # Test paramter
    optimizer = tf.train.AdamOptimizer(learning_rate=lr)
    btcN = True
    
    #Define graph
    TGraph=tf.Graph()
    [layer_outs, error, accuracy, inputs, targets, sop, t_stp] = nnu.net_model(TGraph, train_data, 
                            convParams=cnPs, maxPoolParams=mPPs, kernelParams=kPs, cnNonLinearity=nonLin,
                            fcParams=fcPs, fcNonLinearity=nonLin, optimizer=optimizer, bnCn=btcN)

    
    # Perform test and save results in dictionary
    path = auu.create_fname(testname, [lr, 'batch']) # <<
    start_run = time.time()
    test_error, test_accuracy = run_session(TGraph, t_stp, train_data, valid_data, inputs, targets, path, sop, epochs)
    runtime = round(time.time() - start_run, 2)
    results[str(lr)] = [runtime, test_error, test_accuracy] #<<

    # Print results to console
    print("Runtime: {0}".format(runtime))
    print("Test Error: {0}\nTest Accuracy: {1}".format(test_error, test_accuracy))
    
# save CSV file and print final    
auu.dict2csv(results, '../notebooks/Coursework4/results/', testname)

Runtime: 1481.8
Test Error: 2.679424524307251
Test Accuracy: 0.19680000841617584
Runtime: 1482.43
Test Error: 2.553729295730591
Test Accuracy: 0.2224999964237213
Runtime: 1504.71
Test Error: 2.5977416038513184
Test Accuracy: 0.1962999850511551
Runtime: 1496.81
Test Error: 2.5554988384246826
Test Accuracy: 0.23489999771118164
Runtime: 1350.87
Test Error: 3.543579339981079
Test Accuracy: 0.03999999910593033
Runtime: 1303.88
Test Error: 5.572109222412109
Test Accuracy: 0.03999999910593033


## Mode Tests

In [9]:
epochs=20

nonLin=tf.nn.elu

mode_test_vals = ['default', 'timbre', 'full', 'timbre-deltas']

#### Conv 1D

In [7]:
'''Conv1D'''
testname='conv1d_data_25'
lr = 1e-4

fcPs = [(1500,200),(200,train_data.num_classes)]
cnPs = [(1,10),(10,20),(20,30),(30,40)]
kPs = [(5,1),(5,1),(5,1),(5,1)]
mPPs = [(2,1),(2,1),(2,1),(3,1)]
results={}

for mode in mode_test_vals:
    
    if mode=='default':
        train_data = train_data_default25
        valid_data = valid_data_default25
    elif mode =='timbre':
        train_data = train_data_timbre25
        valid_data = valid_data_timbre25
    elif mode=='full':
        train_data = train_data_full25
        valid_data = valid_data_full25
    else:
        train_data = train_data_timbreDeltas25
        valid_data = valid_data_timbreDeltas25
    
    # Test paramter
    optimizer = tf.train.AdamOptimizer(learning_rate=lr)
    btcN = True
    
    #Define graph
    TGraph=tf.Graph()
    [layer_outs, error, accuracy, inputs, targets, sop, t_stp] = nnu.net_model(TGraph, train_data, 
                            convParams=cnPs, maxPoolParams=mPPs, kernelParams=kPs, cnNonLinearity=nonLin,
                            fcParams=fcPs, fcNonLinearity=nonLin, optimizer=optimizer, bnCn=btcN)

    
    # Perform test and save results in dictionary
    path = auu.create_fname(testname, [mode]) # <<
    start_run = time.time()
    test_error, test_accuracy = run_session(TGraph, t_stp, train_data, valid_data, inputs, targets, path, sop, epochs)
    runtime = round(time.time() - start_run, 2)
    results[mode] = [runtime, test_error, test_accuracy] #<<

    # Print results to console
    print("Runtime: {0}".format(runtime))
    print("Test Error: {0}\nTest Accuracy: {1}".format(test_error, test_accuracy))
    
# save CSV file and print final    
auu.dict2csv(results, '../notebooks/Coursework4/results/', testname)

Runtime: 790.73
Test Error: 2.518949508666992
Test Accuracy: 0.24699999392032623
Runtime: 440.23
Test Error: 2.5674610137939453
Test Accuracy: 0.22710001468658447
Runtime: 2518.65
Test Error: 2.505674123764038
Test Accuracy: 0.24089999496936798
Runtime: 1093.4
Test Error: 2.5510127544403076
Test Accuracy: 0.2329999953508377


#### Con 2D Full Frequency

In [11]:
'''Conv2D Full Frequency'''
testname='conv2dFF_data_25'
lr = 5e-3

fcPs = [(1500,200),(200,train_data.num_classes)]
cnPs = [(1,10),(10,50),(50,100),(100,200)]
mPPs = [(1,1),(2,1),(2,1),(29,1)]
results={}

padding=['VALID', 'SAME', 'SAME', 'SAME']

for mode in mode_test_vals:
    
    if mode=='default':
        train_data = train_data_default25
        valid_data = valid_data_default25
        kPs = [(5,25),(5,1),(5,1),(5,1)]
    elif mode =='timbre':
        train_data = train_data_timbre25
        valid_data = valid_data_timbre25
        kPs = [(5,12),(5,1),(5,1),(5,1)]
    elif mode=='full':
        train_data = train_data_full25
        valid_data = valid_data_full25
        kPs = [(5,88),(5,1),(5,1),(5,1)]
    else:
        train_data = train_data_timbreDeltas25
        valid_data = valid_data_timbreDeltas25
        kPs = [(5,36),(5,1),(5,1),(5,1)]
    
    # Test paramter
    optimizer = tf.train.AdamOptimizer(learning_rate=lr)
    btcN = True
    
    
    #Define graph
    TGraph=tf.Graph()
    [layer_outs, error, accuracy, inputs, targets, sop, t_stp] = nnu.net_model(TGraph, train_data, 
                            convParams=cnPs, maxPoolParams=mPPs, kernelParams=kPs, cnNonLinearity=nonLin,
                            fcParams=fcPs, fcNonLinearity=nonLin, optimizer=optimizer, bnCn=btcN, convPadding=padding)

    
    # Perform test and save results in dictionary
    path = auu.create_fname(testname, [mode]) # <<
    start_run = time.time()
    test_error, test_accuracy = run_session(TGraph, t_stp, train_data, valid_data, inputs, targets, path, sop, epochs)
    runtime = round(time.time() - start_run, 2)
    results[mode] = [runtime, test_error, test_accuracy] #<<

    # Print results to console
    print("Runtime: {0}".format(runtime))
    print("Test Error: {0}\nTest Accuracy: {1}".format(test_error, test_accuracy))
    
# save CSV file and print final    
auu.dict2csv(results, '../notebooks/Coursework4/results/', testname)

Runtime: 307.84
Test Error: 2.6351494789123535
Test Accuracy: 0.2247999906539917
Runtime: 286.05
Test Error: 2.6063339710235596
Test Accuracy: 0.23430000245571136
Runtime: 430.15
Test Error: 2.521533727645874
Test Accuracy: 0.2328999936580658
Runtime: 316.0
Test Error: 2.590006113052368
Test Accuracy: 0.2363000065088272


#### Conv 2D Floating Kernel

In [10]:
'''Conv2D'''
testname='conv2d_data_25'
lr = 5e-3

fcPs = [(1500,200),(200,train_data.num_classes)]
cnPs = [(1,10),(10,30),(30,50),(50,70)]
kPs = [(3,3),(3,3),(3,3),(3,3)]
results={}

for mode in mode_test_vals:
    
    if mode=='default':
        train_data = train_data_default25
        valid_data = valid_data_default25
        mPPs = [(2,1),(2,1),(2,1),(3,5)]
    elif mode =='timbre':
        train_data = train_data_timbre25
        valid_data = valid_data_timbre25
        mPPs = [(2,1),(2,1),(2,1),(3,3)]
    elif mode=='full':
        train_data = train_data_full25
        valid_data = valid_data_full25
        mPPs = [(2,2),(2,2),(2,2),(3,1)]
    else:
        train_data = train_data_timbreDeltas25
        valid_data = valid_data_timbreDeltas25
        mPPs = [(2,1),(2,2),(2,2),(3,1)]
    
    # Test paramter
    optimizer = tf.train.AdamOptimizer(learning_rate=lr)
    btcN = True
    
    #Define graph
    TGraph=tf.Graph()
    [layer_outs, error, accuracy, inputs, targets, sop, t_stp] = nnu.net_model(TGraph, train_data, 
                            convParams=cnPs, maxPoolParams=mPPs, kernelParams=kPs, cnNonLinearity=nonLin,
                            fcParams=fcPs, fcNonLinearity=nonLin, optimizer=optimizer, bnCn=btcN)

    
    # Perform test and save results in dictionary
    path = auu.create_fname(testname, [mode]) # <<
    start_run = time.time()
    test_error, test_accuracy = run_session(TGraph, t_stp, train_data, valid_data, inputs, targets, path, sop, epochs)
    runtime = round(time.time() - start_run, 2)
    results[mode] = [runtime, test_error, test_accuracy] #<<

    # Print results to console
    print("Runtime: {0}".format(runtime))
    print("Test Error: {0}\nTest Accuracy: {1}".format(test_error, test_accuracy))
    
# save CSV file and print final    
auu.dict2csv(results, '../notebooks/Coursework4/results/', testname)

Runtime: 1139.28
Test Error: 2.8131134510040283
Test Accuracy: 0.2263999879360199
Runtime: 669.97
Test Error: 3.221024751663208
Test Accuracy: 0.20319999754428864
Runtime: 1908.85
Test Error: 3.3543500900268555
Test Accuracy: 0.03999999910593033
Runtime: 1211.02
Test Error: 3.858874559402466
Test Accuracy: 0.19670000672340393


# CRNN

In [3]:
def net_crnn(graph, train_data, convParams=None, cnStrides=1, rcParams=None,
            kernelParams=None, maxPoolParams=None, nonL=tf.nn.elu, dropPFc=1, 
            dropPCn=1, bnFc=False, bnCn=False, 
            optimizer=tf.train.AdamOptimizer(learning_rate=5e-6),
            convPadding='SAME'):
    
    with graph.as_default():
        
        layer_outs = []
        
        with tf.name_scope('data'):
            targets=tf.placeholder(tf.float32, [None, train_data.num_classes], name='targets')
            inputs=tf.placeholder(tf.float32, [None, train_data.inputs.shape[1], train_data.inputs.shape[2], train_data.inputs.shape[3]], name='inputs')
            
        conv_outs = nnu.multi_convLayers(inputs, convParams=convParams, cnStrides=cnStrides, kernelParams=kernelParams, maxPoolParams=maxPoolParams,
                                            cnNonLinearity=nonL, dropPCn=dropPCn, bnCn=bnCn, padding=convPadding)
            
        layer_outs.extend(conv_outs)
        
        """
        Recurrent Layer
        """
        
        recurrent_inputs = tf.squeeze(layer_outs[-1], axis=2)
        print(layer_outs[-1].get_shape)
        print(recurrent_inputs.get_shape()[2])
        
        
        weights_in = tf.Variable(tf.random_normal([recurrent_inputs.get_shape().as_list()[2], rcParams[0]]))
        weights_out = tf.Variable(tf.random_normal([rcParams[0], train_data.num_classes]))
        
        biases_in = tf.Variable(tf.zeros([rcParams[0], ])) #<< what is the space
        biases_out = tf.Variable(tf.zeros([train_data.num_classes, ]))
        
        inputs_in = tf.reshape(recurrent_inputs, [-1, recurrent_inputs.get_shape().as_list()[-1]])
        rnn_in = tf.matmul(inputs_in, weights_in) + biases_in
        
        rnn_in = tf.reshape(rnn_in, [-1, recurrent_inputs.get_shape().as_list()[1], rcParams[0]])
        
        lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(rcParams[0], forget_bias=1.0, state_is_tuple=True)
        initial_state = lstm_cell.zero_state(50, dtype=tf.float32) #<< batch size = 50
        
        outputs, final_state = tf.nn.dynamic_rnn(lstm_cell, rnn_in, initial_state =initial_state,
                                                time_major=False)
        
        outputs = tf.unpack(tf.transpose(outputs, [1, 0, 2]))
        prediction = tf.matmul(outputs[-1], weights_out) + biases_out
        
        with tf.name_scope('error'):
            error = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(prediction, targets))
            
        with tf.name_scope('accuracy'):
            accuracy = tf.reduce_mean(tf.cast(
                tf.equal(tf.argmax(prediction, 1), tf.argmax(targets, 1)), 
                tf.float32))
            
        with tf.name_scope('train'):
            gvs = optimizer.compute_gradients(error)
            capped_gvs = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gvs]
            train_step = optimizer.apply_gradients(capped_gvs)
            
        summary_merged = nnu.define_summaries(scalar_sd={'error': error, 'accuracy': accuracy})        
    
    return layer_outs, prediction, error, accuracy, inputs, targets, summary_merged, train_step

In [6]:
train_data_fullNoChromaDeltas = MSD10CustomGenreDataProvider('train', batch_size=50, dimension='full-nochromd')
valid_data_fullNoChromaDeltas = MSD10CustomGenreDataProvider('valid', batch_size=50, dimension='full-nochromd')
print(train_data_fullNoChromaDeltas.inputs.shape)

train_data_fullNoChromaDeltas25 = MSD25CustomGenreDataProvider('train', batch_size=50, dimension='full-nochromd')
valid_data_fullNoChromaDeltas25 = MSD25CustomGenreDataProvider('valid', batch_size=50, dimension='full-nochromd')
print(train_data_fullNoChromaDeltas25.inputs.shape)

print('Data Loaded!')

(40000, 120, 60, 1)
(40000, 120, 60, 1)
Data Loaded!


In [6]:
'''Conv2D'''
testname='conv2d_crnn'
lr = 5e-3
epochs=20
rnn_hidden_layers = [20, 35, 50]

train_data = train_data_fullNoChromaDeltas
valid_data = valid_data_fullNoChromaDeltas

cnPs = [(1,10),(10,30),(30,50),(50,70)]
kPs = [(3,3),(3,3),(3,3),(3,3)]
mPPs = [(2,2),(2,2),(1,5),(1,3)]
results={}

for rc in rnn_hidden_layers:
    
    rcPs = [rc]
    print(rcPs)
    # Test paramter
    optimizer = tf.train.AdamOptimizer(learning_rate=lr)
    btcN = True
    
    #Define graph
    TGraph=tf.Graph()
    
    [layer_outs, prediction, error, accuracy, inputs, targets, sop, t_stp] = net_crnn(TGraph, train_data, 
                            convParams=cnPs, maxPoolParams=mPPs, kernelParams=kPs,
                            rcParams=rcPs, optimizer=optimizer, bnCn=btcN)

    
    # Perform test and save results in dictionary
    path = auu.create_fname(testname, [rc]) # <<
    start_run = time.time()
    test_error, test_accuracy = run_session(TGraph, t_stp, train_data, valid_data, inputs, targets, path, sop, epochs)
    runtime = round(time.time() - start_run, 2)
    results[rc] = [runtime, test_error, test_accuracy] #<<

    # Print results to console
    print("Runtime: {0}".format(runtime))
    print("Test Error: {0}\nTest Accuracy: {1}".format(test_error, test_accuracy))
    
# save CSV file and print final    
auu.dict2csv(results, '../notebooks/Coursework4/results/', testname)

[20]
<bound method Tensor.get_shape of <tf.Tensor 'mp-layer-3/MaxPool:0' shape=(?, 30, 1, 70) dtype=float32>>
70
Runtime: 1510.3
Test Error: 1.3141944408416748
Test Accuracy: 0.5483999848365784
[35]
<bound method Tensor.get_shape of <tf.Tensor 'mp-layer-3/MaxPool:0' shape=(?, 30, 1, 70) dtype=float32>>
70
Runtime: 1512.52
Test Error: 1.3160910606384277
Test Accuracy: 0.5397999286651611
[50]
<bound method Tensor.get_shape of <tf.Tensor 'mp-layer-3/MaxPool:0' shape=(?, 30, 1, 70) dtype=float32>>
70
Runtime: 1508.99
Test Error: 1.5288825035095215
Test Accuracy: 0.4564000070095062


In [7]:
'''Conv2D'''
testname='conv2d_crnn_25'
lr = 5e-3
epochs=20
rnn_hidden_layers = [20, 35, 50]

train_data = train_data_fullNoChromaDeltas25
valid_data = valid_data_fullNoChromaDeltas25

cnPs = [(1,10),(10,30),(30,50),(50,70)]
kPs = [(3,3),(3,3),(3,3),(3,3)]
mPPs = [(2,2),(2,2),(1,5),(1,3)]
results={}

for rc in rnn_hidden_layers:
    
    rcPs = [rc]
    # Test paramter
    optimizer = tf.train.AdamOptimizer(learning_rate=lr)
    btcN = True
    
    #Define graph
    TGraph=tf.Graph()
    
    [layer_outs, prediction, error, accuracy, inputs, targets, sop, t_stp] = net_crnn(TGraph, train_data, 
                            convParams=cnPs, maxPoolParams=mPPs, kernelParams=kPs,
                            rcParams=rcPs, optimizer=optimizer, bnCn=btcN)

    
    # Perform test and save results in dictionary
    path = auu.create_fname(testname, [rc]) # <<
    start_run = time.time()
    test_error, test_accuracy = run_session(TGraph, t_stp, train_data, valid_data, inputs, targets, path, sop, epochs)
    runtime = round(time.time() - start_run, 2)
    results[rc] = [runtime, test_error, test_accuracy] #<<

    # Print results to console
    print("Runtime: {0}".format(runtime))
    print("Test Error: {0}\nTest Accuracy: {1}".format(test_error, test_accuracy))
    
# save CSV file and print final    
auu.dict2csv(results, '../notebooks/Coursework4/results/', testname)

<bound method Tensor.get_shape of <tf.Tensor 'mp-layer-3/MaxPool:0' shape=(?, 30, 1, 70) dtype=float32>>
70
Runtime: 1523.81
Test Error: 2.554560899734497
Test Accuracy: 0.21880000829696655
<bound method Tensor.get_shape of <tf.Tensor 'mp-layer-3/MaxPool:0' shape=(?, 30, 1, 70) dtype=float32>>
70
Runtime: 1521.71
Test Error: 2.569983720779419
Test Accuracy: 0.21570000052452087
<bound method Tensor.get_shape of <tf.Tensor 'mp-layer-3/MaxPool:0' shape=(?, 30, 1, 70) dtype=float32>>
70
Runtime: 1477.06
Test Error: 3.2230424880981445
Test Accuracy: 0.03999999538064003


## Learning rate and dropout

In [5]:
'''Conv2D'''
testname='conv2d_crnn_learning_rate_dropout'
epochs=20
rnn_hidden_layers = [20]

learning_rate_test = [5e-3, 1e-2, 5e-2]
dropout_test = [0.4, 0.6, 0.8, 1]

train_data = train_data_fullNoChromaDeltas
valid_data = valid_data_fullNoChromaDeltas
 
rcPs = rnn_hidden_layers
cnPs = [(1,10),(10,30),(30,50),(50,70)]
kPs = [(3,3),(3,3),(3,3),(3,3)]
mPPs = [(2,2),(2,2),(1,5),(1,3)]
results={}

for lr in learning_rate_test:
    for drp in dropout_test:
    
        # Test paramter
        optimizer = tf.train.AdamOptimizer(learning_rate=lr)
        btcN = True

        #Define graph
        TGraph=tf.Graph()

        [layer_outs, prediction, error, accuracy, inputs, targets, sop, t_stp] = net_crnn(TGraph, train_data, 
                                convParams=cnPs, maxPoolParams=mPPs, kernelParams=kPs,
                                rcParams=rcPs, optimizer=optimizer, bnCn=btcN, dropPCn=drp)


        # Perform test and save results in dictionary
        path = auu.create_fname(testname, [lr, drp]) # <<
        start_run = time.time()
        test_error, test_accuracy = run_session(TGraph, t_stp, train_data, valid_data, inputs, targets, path, sop, epochs)
        runtime = round(time.time() - start_run, 2)
        
        name = str(lr)+'_'+str(drp)
        results[name] = [runtime, test_error, test_accuracy] #<<

        # Print results to console
        print("Runtime: {0}".format(runtime))
        print("Test Error: {0}\nTest Accuracy: {1}".format(test_error, test_accuracy))
    
    
# save CSV file and print final    
auu.dict2csv(results, '../notebooks/Coursework4/results/', testname)

<bound method Tensor.get_shape of <tf.Tensor 'mp-layer-3/MaxPool:0' shape=(?, 30, 1, 70) dtype=float32>>
70
Runtime: 1640.67
Test Error: 2.304070472717285
Test Accuracy: 0.10000000149011612
<bound method Tensor.get_shape of <tf.Tensor 'mp-layer-3/MaxPool:0' shape=(?, 30, 1, 70) dtype=float32>>
70
Runtime: 1666.47
Test Error: 2.3041648864746094
Test Accuracy: 0.10000000149011612
<bound method Tensor.get_shape of <tf.Tensor 'mp-layer-3/MaxPool:0' shape=(?, 30, 1, 70) dtype=float32>>
70
Runtime: 1745.21
Test Error: 1.4133715629577637
Test Accuracy: 0.5031999945640564
<bound method Tensor.get_shape of <tf.Tensor 'mp-layer-3/MaxPool:0' shape=(?, 30, 1, 70) dtype=float32>>
70
Runtime: 1494.36
Test Error: 1.3425441980361938
Test Accuracy: 0.5275999903678894
<bound method Tensor.get_shape of <tf.Tensor 'mp-layer-3/MaxPool:0' shape=(?, 30, 1, 70) dtype=float32>>
70
Runtime: 1690.42
Test Error: 2.304098129272461
Test Accuracy: 0.10000000149011612
<bound method Tensor.get_shape of <tf.Tensor 'mp-

# MultiKernel CNN

In [4]:
# Cell with Function that produces a visual representation of the graph passed using TENSORBOARD.

from IPython.display import display, HTML
import datetime

def show_graph(graph_def, frame_size=(900, 600)):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:{height}px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(height=frame_size[1], data=repr(str(graph_def)), id='graph'+timestamp)
    iframe = """
        <iframe seamless style="width:{width}px;height:{height}px;border:0" srcdoc="{src}"></iframe>
    """.format(width=frame_size[0], height=frame_size[1] + 20, src=code.replace('"', '&quot;'))
    display(HTML(iframe))

In [7]:
'''Conv2D'''
testname='conv2d_multikernel'
epochs=20

train_data = train_data_fullNoChromaDeltas
valid_data = valid_data_fullNoChromaDeltas

fcParams = [(1500,1000),(1000,train_data.num_classes)]
cnPs1 = [(1,5),(5,10)]
cnPs2 = [(1,5),(5,10)]
cnPs3 = [(1,5),(5,10)]
cnPs4 = [(1,5),(5,10)]
cnPsO = [(40,50)]

kPs = [(5,1),(5,1)]
kPsO = [(5,1)]
mPPs = [(2,1),(2,1)]
mPPsO = [(2,1)]
results={}


optimizer = tf.train.AdamOptimizer(learning_rate=1e-4)
btcN = True
TGraph=tf.Graph()    
    
with TGraph.as_default():
    layer_outs = []
        
    '''
    Set placeholder of targets and inputs.
    The inputs placeholder dimensions will be determined by the data.
    '''
    with tf.name_scope('data'):
        targets = tf.placeholder(tf.float32, [None, train_data.num_classes], 'targets')
        inputs = tf.placeholder(tf.float32, [None, train_data.inputs.shape[1], train_data.inputs.shape[2],
                                                 train_data.inputs.shape[3]], name='inputs')
        
        input_shape= inputs.get_shape().as_list()
        print(input_shape)
        inputs1 = tf.slice(inputs, [0,0,0,0], [50, input_shape[1], 12, input_shape[3]]) #timbre
        inputs2 = tf.slice(inputs, [0,0,15,0], [50, input_shape[1], 12, input_shape[3]]) #deltas
        inputs3 = tf.slice(inputs, [0,0,30,0], [50, input_shape[1], 12, input_shape[3]]) #deltadeltas
        inputs4 = tf.slice(inputs, [0,0,45,0], [50, input_shape[1], 12, input_shape[3]]) #chroma
        
        '''
        Create Convolutional Layers
        '''
        conv_outs1 = nnu.multi_convLayers(inputs1, convParams=cnPs1, cnStrides=1, kernelParams=kPs, 
                                        maxPoolParams=mPPs, cnNonLinearity=tf.nn.elu, dropPCn=1, 
                                        bnCn=btcN, padding='SAME')
            
        conv_outs2 = nnu.multi_convLayers(inputs2, convParams=cnPs2, cnStrides=1, kernelParams=kPs, 
                                        maxPoolParams=mPPs, cnNonLinearity=tf.nn.elu, dropPCn=1, 
                                        bnCn=btcN, padding='SAME')
        
        conv_outs3 = nnu.multi_convLayers(inputs3, convParams=cnPs3, cnStrides=1, kernelParams=kPs, 
                                        maxPoolParams=mPPs, cnNonLinearity=tf.nn.elu, dropPCn=1, 
                                        bnCn=btcN, padding='SAME')   
            
        conv_outs4 = nnu.multi_convLayers(inputs4, convParams=cnPs4, cnStrides=1, kernelParams=kPs, 
                                        maxPoolParams=mPPs, cnNonLinearity=tf.nn.elu, dropPCn=1, 
                                        bnCn=btcN, padding='SAME')      
        
        
        ensemble_out = tf.concat(3, [conv_outs1[-1], conv_outs2[-1]])
        ensemble_out = tf.concat(3, [ensemble_out, conv_outs3[-1]])
        ensemble_out = tf.concat(3, [ensemble_out, conv_outs4[-1]])
            
        conv_outsO = nnu.multi_convLayers(ensemble_out, convParams=cnPsO, cnStrides=1, kernelParams=kPsO, 
                                        maxPoolParams=mPPsO, cnNonLinearity=tf.nn.elu, dropPCn=1, 
                                        bnCn=btcN, padding='SAME')  
        
        shape= conv_outsO[-1].get_shape().as_list()
        print(shape)
        fcParams[0] = (shape[1]*shape[2]*shape[3], fcParams[0][1])
        print(fcParams[0])
        print(conv_outsO[-1])
        input_data = tf.reshape(conv_outsO[-1], [-1, fcParams[0][0]])
        print(input_data)
        
        fc_outs = nnu.multi_fcLayers(input_data, fcParams=fcParams[:-1], fcNonLinearity=tf.nn.elu, dropPFc=1, bnFc=False)
        print(fc_outs[-1])
        layer_outs.extend(fc_outs)
        input_data=layer_outs[-1]
        
        with tf.name_scope('output-layer'):
            output = nnu.fully_connected_layer(input_data, fcParams[-1], nonlinearity=tf.identity)
            layer_outs.append(output)
            
                            
        # Define the operations for the error
        with tf.name_scope('error'):   
            error = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits(layer_outs[-1], targets))

        # Define the function to calculate the accuracy of the system
        with tf.name_scope('accuracy'):
            accuracy = tf.reduce_mean(tf.cast(
                tf.equal(tf.argmax(layer_outs[-1], 1), tf.argmax(targets, 1)), 
                tf.float32))
        
        # Define optimizer and train step
        with tf.name_scope('train'):
            gvs = optimizer.compute_gradients(error)
            capped_gvs = [(tf.clip_by_value(grad, -2., 2.), var) for grad, var in gvs]
            train_step = optimizer.apply_gradients(capped_gvs)
                
        # Define summaries        
        summary_merged = nnu.define_summaries(scalar_sd={'error': error, 'accuracy': accuracy})        
        
#show_graph(TGraph)        


# Perform test and save results in dictionary
path = auu.create_fname(testname, ['mk1']) # <<
start_run = time.time()
test_error, test_accuracy = run_session(TGraph, train_step, train_data, valid_data, inputs, targets, path, summary_merged, epochs)
runtime = round(time.time() - start_run, 2)
results['mk1'] = [runtime, test_error, test_accuracy] #<<

# Print results to console
print("Runtime: {0}".format(runtime))
print("Test Error: {0}\nTest Accuracy: {1}".format(test_error, test_accuracy))
    
    
# save CSV file and print final    
auu.dict2csv(results, '../notebooks/Coursework4/results/', testname)

[None, 120, 60, 1]
[50, 15, 12, 40]
(7200, 4500)
Tensor("data/mp-layer-0_4/MaxPool:0", shape=(50, 15, 12, 40), dtype=float32)
Tensor("data/Reshape:0", shape=(50, 7200), dtype=float32)
Tensor("data/fc-layer-0/Elu:0", shape=(50, 4500), dtype=float32)


ResourceExhaustedError: OOM when allocating tensor with shape[7200,4500]
	 [[Node: data/fc-layer-0/truncated_normal/TruncatedNormal = TruncatedNormal[T=DT_INT32, dtype=DT_FLOAT, seed=0, seed2=0, _device="/job:localhost/replica:0/task:0/gpu:0"](data/fc-layer-0/truncated_normal/shape)]]

Caused by op 'data/fc-layer-0/truncated_normal/TruncatedNormal', defined at:
  File "C:\Users\Mark\Anaconda3\envs\mlp35\lib\runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Mark\Anaconda3\envs\mlp35\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Mark\Anaconda3\envs\mlp35\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "C:\Users\Mark\Anaconda3\envs\mlp35\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\Mark\Anaconda3\envs\mlp35\lib\site-packages\ipykernel\kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().st

In [8]:
'''Conv2D'''
testname='conv1d_multikernel_2'
epochs=20

train_data = train_data_fullNoChromaDeltas
valid_data = valid_data_fullNoChromaDeltas

fcParams = [(1500,1000),(1000,train_data.num_classes)]
cnPs1 = [(1,5),(5,10)]
cnPs2 = [(1,5),(5,10)]
cnPs3 = [(1,5),(5,10)]
cnPs4 = [(1,5),(5,10)]
cnPsO = [(40,50)]

kPs = [(5,1),(5,1)]
kPsO = [(5,1)]
mPPs = [(2,1),(2,1)]
mPPsO = [(2,1)]
results={}


for sz in [6,8,10,12]:
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-4)
    btcN = True
    TGraph=tf.Graph()    

    with TGraph.as_default():
        layer_outs = []

        '''
        Set placeholder of targets and inputs.
        The inputs placeholder dimensions will be determined by the data.
        '''
        with tf.name_scope('data'):
            targets = tf.placeholder(tf.float32, [None, train_data.num_classes], 'targets')
            inputs = tf.placeholder(tf.float32, [None, train_data.inputs.shape[1], train_data.inputs.shape[2],
                                                     train_data.inputs.shape[3]], name='inputs')

            input_shape= inputs.get_shape().as_list()
            print(input_shape)
            inputs1 = tf.slice(inputs, [0,0,0,0], [50, input_shape[1], sz, input_shape[3]]) #timbre
            inputs2 = tf.slice(inputs, [0,0,15,0], [50, input_shape[1], sz, input_shape[3]]) #deltas
            inputs3 = tf.slice(inputs, [0,0,30,0], [50, input_shape[1], sz, input_shape[3]]) #deltadeltas
            inputs4 = tf.slice(inputs, [0,0,45,0], [50, input_shape[1], sz, input_shape[3]]) #chroma

            '''
            Create Convolutional Layers
            '''
            conv_outs1 = nnu.multi_convLayers(inputs1, convParams=cnPs1, cnStrides=1, kernelParams=kPs, 
                                            maxPoolParams=mPPs, cnNonLinearity=tf.nn.elu, dropPCn=1, 
                                            bnCn=btcN, padding='SAME')

            conv_outs2 = nnu.multi_convLayers(inputs2, convParams=cnPs2, cnStrides=1, kernelParams=kPs, 
                                            maxPoolParams=mPPs, cnNonLinearity=tf.nn.elu, dropPCn=1, 
                                            bnCn=btcN, padding='SAME')

            conv_outs3 = nnu.multi_convLayers(inputs3, convParams=cnPs3, cnStrides=1, kernelParams=kPs, 
                                            maxPoolParams=mPPs, cnNonLinearity=tf.nn.elu, dropPCn=1, 
                                            bnCn=btcN, padding='SAME')   

            conv_outs4 = nnu.multi_convLayers(inputs4, convParams=cnPs4, cnStrides=1, kernelParams=kPs, 
                                            maxPoolParams=mPPs, cnNonLinearity=tf.nn.elu, dropPCn=1, 
                                            bnCn=btcN, padding='SAME')      


            ensemble_out = tf.concat(3, [conv_outs1[-1], conv_outs2[-1]])
            ensemble_out = tf.concat(3, [ensemble_out, conv_outs3[-1]])
            ensemble_out = tf.concat(3, [ensemble_out, conv_outs4[-1]])

            conv_outsO = nnu.multi_convLayers(ensemble_out, convParams=cnPsO, cnStrides=1, kernelParams=kPsO, 
                                            maxPoolParams=mPPsO, cnNonLinearity=tf.nn.elu, dropPCn=1, 
                                            bnCn=btcN, padding='SAME')  

            shape= conv_outsO[-1].get_shape().as_list()
            print(shape)
            fcParams[0] = (shape[1]*shape[2]*shape[3], fcParams[0][1])
            print(fcParams[0])
            print(conv_outsO[-1])
            input_data = tf.reshape(conv_outsO[-1], [-1, fcParams[0][0]])
            print(input_data)

            fc_outs = nnu.multi_fcLayers(input_data, fcParams=fcParams[:-1], fcNonLinearity=tf.nn.elu, dropPFc=1, bnFc=False)
            print(fc_outs[-1])
            layer_outs.extend(fc_outs)
            input_data=layer_outs[-1]

            with tf.name_scope('output-layer'):
                output = nnu.fully_connected_layer(input_data, fcParams[-1], nonlinearity=tf.identity)
                layer_outs.append(output)


            # Define the operations for the error
            with tf.name_scope('error'):   
                error = tf.reduce_mean(
                    tf.nn.softmax_cross_entropy_with_logits(layer_outs[-1], targets))

            # Define the function to calculate the accuracy of the system
            with tf.name_scope('accuracy'):
                accuracy = tf.reduce_mean(tf.cast(
                    tf.equal(tf.argmax(layer_outs[-1], 1), tf.argmax(targets, 1)), 
                    tf.float32))

            # Define optimizer and train step
            with tf.name_scope('train'):
                gvs = optimizer.compute_gradients(error)
                capped_gvs = [(tf.clip_by_value(grad, -2., 2.), var) for grad, var in gvs]
                train_step = optimizer.apply_gradients(capped_gvs)

            # Define summaries        
            summary_merged = nnu.define_summaries(scalar_sd={'error': error, 'accuracy': accuracy})        

    #show_graph(TGraph)        


    # Perform test and save results in dictionary
    path = auu.create_fname(testname, [sz]) # <<
    start_run = time.time()
    test_error, test_accuracy = run_session(TGraph, train_step, train_data, valid_data, inputs, targets, path, summary_merged, epochs)
    runtime = round(time.time() - start_run, 2)
    results[sz] = [runtime, test_error, test_accuracy] #<<

    # Print results to console
    print("Runtime: {0}".format(runtime))
    print("Test Error: {0}\nTest Accuracy: {1}".format(test_error, test_accuracy))
    
    
# save CSV file and print final    
auu.dict2csv(results, '../notebooks/Coursework4/results/', testname)

[None, 120, 60, 1]
[50, 15, 6, 50]
(4500, 1000)
Tensor("data/mp-layer-0_4/MaxPool:0", shape=(50, 15, 6, 50), dtype=float32)
Tensor("data/Reshape:0", shape=(50, 4500), dtype=float32)
Tensor("data/fc-layer-0/Elu:0", shape=(50, 1000), dtype=float32)
Runtime: 659.25
Test Error: 1.4037420749664307
Test Accuracy: 0.5006999969482422
[None, 120, 60, 1]
[50, 15, 8, 50]
(6000, 1000)
Tensor("data/mp-layer-0_4/MaxPool:0", shape=(50, 15, 8, 50), dtype=float32)
Tensor("data/Reshape:0", shape=(50, 6000), dtype=float32)
Tensor("data/fc-layer-0/Elu:0", shape=(50, 1000), dtype=float32)
Runtime: 823.45
Test Error: 1.3438912630081177
Test Accuracy: 0.521899938583374
[None, 120, 60, 1]
[50, 15, 10, 50]
(7500, 1000)
Tensor("data/mp-layer-0_4/MaxPool:0", shape=(50, 15, 10, 50), dtype=float32)
Tensor("data/Reshape:0", shape=(50, 7500), dtype=float32)
Tensor("data/fc-layer-0/Elu:0", shape=(50, 1000), dtype=float32)
Runtime: 1005.11
Test Error: 1.3134313821792603
Test Accuracy: 0.5399999618530273
[None, 120, 60

In [9]:
'''Conv2D'''
testname='conv2d_multikernel_2'
epochs=20

train_data = train_data_fullNoChromaDeltas
valid_data = valid_data_fullNoChromaDeltas

fcParams = [(1500,1000),(1000,train_data.num_classes)]
cnPs1 = [(1,5),(5,10)]
cnPs2 = [(1,5),(5,10)]
cnPs3 = [(1,5),(5,10)]
cnPs4 = [(1,5),(5,10)]
cnPsO = [(40,50)]

kPs = [(3,3),(3,3)]
kPsO = [(3,3)]
mPPs = [(1,1),(2,1)]
mPPsO = [(2,2)]
results={}


for sz in [6,8,10,12]:
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-4)
    btcN = True
    TGraph=tf.Graph()    

    with TGraph.as_default():
        layer_outs = []

        '''
        Set placeholder of targets and inputs.
        The inputs placeholder dimensions will be determined by the data.
        '''
        with tf.name_scope('data'):
            targets = tf.placeholder(tf.float32, [None, train_data.num_classes], 'targets')
            inputs = tf.placeholder(tf.float32, [None, train_data.inputs.shape[1], train_data.inputs.shape[2],
                                                     train_data.inputs.shape[3]], name='inputs')

            input_shape= inputs.get_shape().as_list()
            print(input_shape)
            inputs1 = tf.slice(inputs, [0,0,0,0], [50, input_shape[1], sz, input_shape[3]]) #timbre
            inputs2 = tf.slice(inputs, [0,0,15,0], [50, input_shape[1], sz, input_shape[3]]) #deltas
            inputs3 = tf.slice(inputs, [0,0,30,0], [50, input_shape[1], sz, input_shape[3]]) #deltadeltas
            inputs4 = tf.slice(inputs, [0,0,45,0], [50, input_shape[1], sz, input_shape[3]]) #chroma

            '''
            Create Convolutional Layers
            '''
            conv_outs1 = nnu.multi_convLayers(inputs1, convParams=cnPs1, cnStrides=1, kernelParams=kPs, 
                                            maxPoolParams=mPPs, cnNonLinearity=tf.nn.elu, dropPCn=1, 
                                            bnCn=btcN, padding='SAME')

            conv_outs2 = nnu.multi_convLayers(inputs2, convParams=cnPs2, cnStrides=1, kernelParams=kPs, 
                                            maxPoolParams=mPPs, cnNonLinearity=tf.nn.elu, dropPCn=1, 
                                            bnCn=btcN, padding='SAME')

            conv_outs3 = nnu.multi_convLayers(inputs3, convParams=cnPs3, cnStrides=1, kernelParams=kPs, 
                                            maxPoolParams=mPPs, cnNonLinearity=tf.nn.elu, dropPCn=1, 
                                            bnCn=btcN, padding='SAME')   

            conv_outs4 = nnu.multi_convLayers(inputs4, convParams=cnPs4, cnStrides=1, kernelParams=kPs, 
                                            maxPoolParams=mPPs, cnNonLinearity=tf.nn.elu, dropPCn=1, 
                                            bnCn=btcN, padding='SAME')      


            ensemble_out = tf.concat(3, [conv_outs1[-1], conv_outs2[-1]])
            ensemble_out = tf.concat(3, [ensemble_out, conv_outs3[-1]])
            ensemble_out = tf.concat(3, [ensemble_out, conv_outs4[-1]])

            conv_outsO = nnu.multi_convLayers(ensemble_out, convParams=cnPsO, cnStrides=1, kernelParams=kPsO, 
                                            maxPoolParams=mPPsO, cnNonLinearity=tf.nn.elu, dropPCn=1, 
                                            bnCn=btcN, padding='SAME')  

            shape= conv_outsO[-1].get_shape().as_list()
            print(shape)
            fcParams[0] = (shape[1]*shape[2]*shape[3], fcParams[0][1])
            print(fcParams[0])
            print(conv_outsO[-1])
            input_data = tf.reshape(conv_outsO[-1], [-1, fcParams[0][0]])
            print(input_data)

            fc_outs = nnu.multi_fcLayers(input_data, fcParams=fcParams[:-1], fcNonLinearity=tf.nn.elu, dropPFc=1, bnFc=False)
            print(fc_outs[-1])
            layer_outs.extend(fc_outs)
            input_data=layer_outs[-1]

            with tf.name_scope('output-layer'):
                output = nnu.fully_connected_layer(input_data, fcParams[-1], nonlinearity=tf.identity)
                layer_outs.append(output)


            # Define the operations for the error
            with tf.name_scope('error'):   
                error = tf.reduce_mean(
                    tf.nn.softmax_cross_entropy_with_logits(layer_outs[-1], targets))

            # Define the function to calculate the accuracy of the system
            with tf.name_scope('accuracy'):
                accuracy = tf.reduce_mean(tf.cast(
                    tf.equal(tf.argmax(layer_outs[-1], 1), tf.argmax(targets, 1)), 
                    tf.float32))

            # Define optimizer and train step
            with tf.name_scope('train'):
                gvs = optimizer.compute_gradients(error)
                capped_gvs = [(tf.clip_by_value(grad, -2., 2.), var) for grad, var in gvs]
                train_step = optimizer.apply_gradients(capped_gvs)

            # Define summaries        
            summary_merged = nnu.define_summaries(scalar_sd={'error': error, 'accuracy': accuracy})        

    #show_graph(TGraph)        


    # Perform test and save results in dictionary
    path = auu.create_fname(testname, [sz]) # <<
    start_run = time.time()
    test_error, test_accuracy = run_session(TGraph, train_step, train_data, valid_data, inputs, targets, path, summary_merged, epochs)
    runtime = round(time.time() - start_run, 2)
    results[sz] = [runtime, test_error, test_accuracy] #<<

    # Print results to console
    print("Runtime: {0}".format(runtime))
    print("Test Error: {0}\nTest Accuracy: {1}".format(test_error, test_accuracy))
    
    
# save CSV file and print final    
auu.dict2csv(results, '../notebooks/Coursework4/results/', testname)

[None, 120, 60, 1]
[50, 30, 3, 50]
(4500, 1000)
Tensor("data/mp-layer-0_4/MaxPool:0", shape=(50, 30, 3, 50), dtype=float32)
Tensor("data/Reshape:0", shape=(50, 4500), dtype=float32)
Tensor("data/fc-layer-0/Elu:0", shape=(50, 1000), dtype=float32)
Runtime: 899.86
Test Error: 1.5024868249893188
Test Accuracy: 0.46640002727508545
[None, 120, 60, 1]
[50, 30, 4, 50]
(6000, 1000)
Tensor("data/mp-layer-0_4/MaxPool:0", shape=(50, 30, 4, 50), dtype=float32)
Tensor("data/Reshape:0", shape=(50, 6000), dtype=float32)
Tensor("data/fc-layer-0/Elu:0", shape=(50, 1000), dtype=float32)
Runtime: 1112.0
Test Error: 1.4614328145980835
Test Accuracy: 0.4791000485420227
[None, 120, 60, 1]
[50, 30, 5, 50]
(7500, 1000)
Tensor("data/mp-layer-0_4/MaxPool:0", shape=(50, 30, 5, 50), dtype=float32)
Tensor("data/Reshape:0", shape=(50, 7500), dtype=float32)
Tensor("data/fc-layer-0/Elu:0", shape=(50, 1000), dtype=float32)
Runtime: 1342.14
Test Error: 1.5275181531906128
Test Accuracy: 0.44700005650520325
[None, 120, 6

In [10]:
'''Conv2D'''
testname='conv2dff_multikernel_2'
epochs=20

train_data = train_data_fullNoChromaDeltas
valid_data = valid_data_fullNoChromaDeltas

fcParams = [(1500,1000),(1000,train_data.num_classes)]
cnPs1 = [(1,5),(5,10)]
cnPs2 = [(1,5),(5,10)]
cnPs3 = [(1,5),(5,10)]
cnPs4 = [(1,5),(5,10)]
cnPsO = [(40,50)]


kPsO = [(3,3)]
mPPs = [(1,1),(2,1)]
mPPsO = [(59,1)]
results={}


for sz in [6,8,10,12]:
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-4)
    btcN = True
    TGraph=tf.Graph()    
    
    kPs = [(3,sz),(3,1)]
    
    with TGraph.as_default():
        layer_outs = []

        '''
        Set placeholder of targets and inputs.
        The inputs placeholder dimensions will be determined by the data.
        '''
        with tf.name_scope('data'):
            targets = tf.placeholder(tf.float32, [None, train_data.num_classes], 'targets')
            inputs = tf.placeholder(tf.float32, [None, train_data.inputs.shape[1], train_data.inputs.shape[2],
                                                     train_data.inputs.shape[3]], name='inputs')

            input_shape= inputs.get_shape().as_list()
            print(input_shape)
            inputs1 = tf.slice(inputs, [0,0,0,0], [50, input_shape[1], sz, input_shape[3]]) #timbre
            inputs2 = tf.slice(inputs, [0,0,15,0], [50, input_shape[1], sz, input_shape[3]]) #deltas
            inputs3 = tf.slice(inputs, [0,0,30,0], [50, input_shape[1], sz, input_shape[3]]) #deltadeltas
            inputs4 = tf.slice(inputs, [0,0,45,0], [50, input_shape[1], sz, input_shape[3]]) #chroma

            '''
            Create Convolutional Layers
            '''
            conv_outs1 = nnu.multi_convLayers(inputs1, convParams=cnPs1, cnStrides=1, kernelParams=kPs, 
                                            maxPoolParams=mPPs, cnNonLinearity=tf.nn.elu, dropPCn=1, 
                                            bnCn=btcN, padding=['VALID', 'SAME'])

            conv_outs2 = nnu.multi_convLayers(inputs2, convParams=cnPs2, cnStrides=1, kernelParams=kPs, 
                                            maxPoolParams=mPPs, cnNonLinearity=tf.nn.elu, dropPCn=1, 
                                            bnCn=btcN, padding=['VALID', 'SAME'])

            conv_outs3 = nnu.multi_convLayers(inputs3, convParams=cnPs3, cnStrides=1, kernelParams=kPs, 
                                            maxPoolParams=mPPs, cnNonLinearity=tf.nn.elu, dropPCn=1, 
                                            bnCn=btcN, padding=['VALID', 'SAME'])   

            conv_outs4 = nnu.multi_convLayers(inputs4, convParams=cnPs4, cnStrides=1, kernelParams=kPs, 
                                            maxPoolParams=mPPs, cnNonLinearity=tf.nn.elu, dropPCn=1, 
                                            bnCn=btcN, padding=['VALID', 'SAME'])      


            ensemble_out = tf.concat(3, [conv_outs1[-1], conv_outs2[-1]])
            ensemble_out = tf.concat(3, [ensemble_out, conv_outs3[-1]])
            ensemble_out = tf.concat(3, [ensemble_out, conv_outs4[-1]])

            conv_outsO = nnu.multi_convLayers(ensemble_out, convParams=cnPsO, cnStrides=1, kernelParams=kPsO, 
                                            maxPoolParams=mPPsO, cnNonLinearity=tf.nn.elu, dropPCn=1, 
                                            bnCn=btcN, padding='SAME')  

            shape= conv_outsO[-1].get_shape().as_list()
            print(shape)
            fcParams[0] = (shape[1]*shape[2]*shape[3], fcParams[0][1])
            print(fcParams[0])
            print(conv_outsO[-1])
            input_data = tf.reshape(conv_outsO[-1], [-1, fcParams[0][0]])
            print(input_data)

            fc_outs = nnu.multi_fcLayers(input_data, fcParams=fcParams[:-1], fcNonLinearity=tf.nn.elu, dropPFc=1, bnFc=False)
            print(fc_outs[-1])
            layer_outs.extend(fc_outs)
            input_data=layer_outs[-1]

            with tf.name_scope('output-layer'):
                output = nnu.fully_connected_layer(input_data, fcParams[-1], nonlinearity=tf.identity)
                layer_outs.append(output)


            # Define the operations for the error
            with tf.name_scope('error'):   
                error = tf.reduce_mean(
                    tf.nn.softmax_cross_entropy_with_logits(layer_outs[-1], targets))

            # Define the function to calculate the accuracy of the system
            with tf.name_scope('accuracy'):
                accuracy = tf.reduce_mean(tf.cast(
                    tf.equal(tf.argmax(layer_outs[-1], 1), tf.argmax(targets, 1)), 
                    tf.float32))

            # Define optimizer and train step
            with tf.name_scope('train'):
                gvs = optimizer.compute_gradients(error)
                capped_gvs = [(tf.clip_by_value(grad, -2., 2.), var) for grad, var in gvs]
                train_step = optimizer.apply_gradients(capped_gvs)

            # Define summaries        
            summary_merged = nnu.define_summaries(scalar_sd={'error': error, 'accuracy': accuracy})        

    #show_graph(TGraph)        


    # Perform test and save results in dictionary
    path = auu.create_fname(testname, [sz]) # <<
    start_run = time.time()
    test_error, test_accuracy = run_session(TGraph, train_step, train_data, valid_data, inputs, targets, path, summary_merged, epochs)
    runtime = round(time.time() - start_run, 2)
    results[sz] = [runtime, test_error, test_accuracy] #<<

    # Print results to console
    print("Runtime: {0}".format(runtime))
    print("Test Error: {0}\nTest Accuracy: {1}".format(test_error, test_accuracy))
    
    
# save CSV file and print final    
auu.dict2csv(results, '../notebooks/Coursework4/results/', testname)

[None, 120, 60, 1]
[50, 1, 1, 50]
(50, 1000)
Tensor("data/mp-layer-0_4/MaxPool:0", shape=(50, 1, 1, 50), dtype=float32)
Tensor("data/Reshape:0", shape=(50, 50), dtype=float32)
Tensor("data/fc-layer-0/Elu:0", shape=(50, 1000), dtype=float32)
Runtime: 339.76
Test Error: 1.6028882265090942
Test Accuracy: 0.43090003728866577
[None, 120, 60, 1]
[50, 1, 1, 50]
(50, 1000)
Tensor("data/mp-layer-0_4/MaxPool:0", shape=(50, 1, 1, 50), dtype=float32)
Tensor("data/Reshape:0", shape=(50, 50), dtype=float32)
Tensor("data/fc-layer-0/Elu:0", shape=(50, 1000), dtype=float32)
Runtime: 316.26
Test Error: 1.6017009019851685
Test Accuracy: 0.43459999561309814
[None, 120, 60, 1]
[50, 1, 1, 50]
(50, 1000)
Tensor("data/mp-layer-0_4/MaxPool:0", shape=(50, 1, 1, 50), dtype=float32)
Tensor("data/Reshape:0", shape=(50, 50), dtype=float32)
Tensor("data/fc-layer-0/Elu:0", shape=(50, 1000), dtype=float32)
Runtime: 318.36
Test Error: 1.6103527545928955
Test Accuracy: 0.4351000189781189
[None, 120, 60, 1]
[50, 1, 1, 50

In [6]:
'''Conv2D'''
testname='conv1d_multikernel_2_25'
epochs=20

train_data = train_data_fullNoChromaDeltas25
valid_data = valid_data_fullNoChromaDeltas25

fcParams = [(1500,1000),(1000,train_data.num_classes)]
cnPs1 = [(1,5),(5,10)]
cnPs2 = [(1,5),(5,10)]
cnPs3 = [(1,5),(5,10)]
cnPs4 = [(1,5),(5,10)]
cnPsO = [(40,50)]

kPs = [(5,1),(5,1)]
kPsO = [(5,1)]
mPPs = [(2,1),(2,1)]
mPPsO = [(2,1)]
results={}


for sz in [6,8,10,12]:
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-4)
    btcN = True
    TGraph=tf.Graph()    

    with TGraph.as_default():
        layer_outs = []

        '''
        Set placeholder of targets and inputs.
        The inputs placeholder dimensions will be determined by the data.
        '''
        with tf.name_scope('data'):
            targets = tf.placeholder(tf.float32, [None, train_data.num_classes], 'targets')
            inputs = tf.placeholder(tf.float32, [None, train_data.inputs.shape[1], train_data.inputs.shape[2],
                                                     train_data.inputs.shape[3]], name='inputs')

            input_shape= inputs.get_shape().as_list()
            print(input_shape)
            inputs1 = tf.slice(inputs, [0,0,0,0], [50, input_shape[1], sz, input_shape[3]]) #timbre
            inputs2 = tf.slice(inputs, [0,0,15,0], [50, input_shape[1], sz, input_shape[3]]) #deltas
            inputs3 = tf.slice(inputs, [0,0,30,0], [50, input_shape[1], sz, input_shape[3]]) #deltadeltas
            inputs4 = tf.slice(inputs, [0,0,45,0], [50, input_shape[1], sz, input_shape[3]]) #chroma

            '''
            Create Convolutional Layers
            '''
            conv_outs1 = nnu.multi_convLayers(inputs1, convParams=cnPs1, cnStrides=1, kernelParams=kPs, 
                                            maxPoolParams=mPPs, cnNonLinearity=tf.nn.elu, dropPCn=1, 
                                            bnCn=btcN, padding='SAME')

            conv_outs2 = nnu.multi_convLayers(inputs2, convParams=cnPs2, cnStrides=1, kernelParams=kPs, 
                                            maxPoolParams=mPPs, cnNonLinearity=tf.nn.elu, dropPCn=1, 
                                            bnCn=btcN, padding='SAME')

            conv_outs3 = nnu.multi_convLayers(inputs3, convParams=cnPs3, cnStrides=1, kernelParams=kPs, 
                                            maxPoolParams=mPPs, cnNonLinearity=tf.nn.elu, dropPCn=1, 
                                            bnCn=btcN, padding='SAME')   

            conv_outs4 = nnu.multi_convLayers(inputs4, convParams=cnPs4, cnStrides=1, kernelParams=kPs, 
                                            maxPoolParams=mPPs, cnNonLinearity=tf.nn.elu, dropPCn=1, 
                                            bnCn=btcN, padding='SAME')      


            ensemble_out = tf.concat(3, [conv_outs1[-1], conv_outs2[-1]])
            ensemble_out = tf.concat(3, [ensemble_out, conv_outs3[-1]])
            ensemble_out = tf.concat(3, [ensemble_out, conv_outs4[-1]])

            conv_outsO = nnu.multi_convLayers(ensemble_out, convParams=cnPsO, cnStrides=1, kernelParams=kPsO, 
                                            maxPoolParams=mPPsO, cnNonLinearity=tf.nn.elu, dropPCn=1, 
                                            bnCn=btcN, padding='SAME')  

            shape= conv_outsO[-1].get_shape().as_list()
            print(shape)
            fcParams[0] = (shape[1]*shape[2]*shape[3], fcParams[0][1])
            print(fcParams[0])
            print(conv_outsO[-1])
            input_data = tf.reshape(conv_outsO[-1], [-1, fcParams[0][0]])
            print(input_data)

            fc_outs = nnu.multi_fcLayers(input_data, fcParams=fcParams[:-1], fcNonLinearity=tf.nn.elu, dropPFc=1, bnFc=False)
            print(fc_outs[-1])
            layer_outs.extend(fc_outs)
            input_data=layer_outs[-1]

            with tf.name_scope('output-layer'):
                output = nnu.fully_connected_layer(input_data, fcParams[-1], nonlinearity=tf.identity)
                layer_outs.append(output)


            # Define the operations for the error
            with tf.name_scope('error'):   
                error = tf.reduce_mean(
                    tf.nn.softmax_cross_entropy_with_logits(layer_outs[-1], targets))

            # Define the function to calculate the accuracy of the system
            with tf.name_scope('accuracy'):
                accuracy = tf.reduce_mean(tf.cast(
                    tf.equal(tf.argmax(layer_outs[-1], 1), tf.argmax(targets, 1)), 
                    tf.float32))

            # Define optimizer and train step
            with tf.name_scope('train'):
                gvs = optimizer.compute_gradients(error)
                capped_gvs = [(tf.clip_by_value(grad, -2., 2.), var) for grad, var in gvs]
                train_step = optimizer.apply_gradients(capped_gvs)

            # Define summaries        
            summary_merged = nnu.define_summaries(scalar_sd={'error': error, 'accuracy': accuracy})        

    #show_graph(TGraph)        


    # Perform test and save results in dictionary
    path = auu.create_fname(testname, [sz]) # <<
    start_run = time.time()
    test_error, test_accuracy = run_session(TGraph, train_step, train_data, valid_data, inputs, targets, path, summary_merged, epochs)
    runtime = round(time.time() - start_run, 2)
    results[sz] = [runtime, test_error, test_accuracy] #<<

    # Print results to console
    print("Runtime: {0}".format(runtime))
    print("Test Error: {0}\nTest Accuracy: {1}".format(test_error, test_accuracy))
    
    
# save CSV file and print final    
auu.dict2csv(results, '../notebooks/Coursework4/results/', testname)

[None, 120, 60, 1]
[50, 15, 6, 50]
(4500, 1000)
Tensor("data/mp-layer-0_4/MaxPool:0", shape=(50, 15, 6, 50), dtype=float32)
Tensor("data/Reshape:0", shape=(50, 4500), dtype=float32)
Tensor("data/fc-layer-0/Elu:0", shape=(50, 1000), dtype=float32)
Runtime: 650.97
Test Error: 2.586364507675171
Test Accuracy: 0.2239999920129776
[None, 120, 60, 1]
[50, 15, 8, 50]
(6000, 1000)
Tensor("data/mp-layer-0_4/MaxPool:0", shape=(50, 15, 8, 50), dtype=float32)
Tensor("data/Reshape:0", shape=(50, 6000), dtype=float32)
Tensor("data/fc-layer-0/Elu:0", shape=(50, 1000), dtype=float32)
Runtime: 770.37
Test Error: 2.5254673957824707
Test Accuracy: 0.2329999953508377
[None, 120, 60, 1]
[50, 15, 10, 50]
(7500, 1000)
Tensor("data/mp-layer-0_4/MaxPool:0", shape=(50, 15, 10, 50), dtype=float32)
Tensor("data/Reshape:0", shape=(50, 7500), dtype=float32)
Tensor("data/fc-layer-0/Elu:0", shape=(50, 1000), dtype=float32)
Runtime: 918.62
Test Error: 2.5207605361938477
Test Accuracy: 0.2370000034570694
[None, 120, 60,

In [35]:
a = tf.constant([[[[1],[2],[3],[4],[5],[6]], [[1],[2],[3],[4],[5],[6]], [[1],[2],[3],[4],[5],[6]], [[1],[2],[3],[4],[5],[6]]]])
print(a)

b = tf.slice(a, [0,0,2,0], [, a.get_shape().as_list()[1], 2, a.get_shape().as_list()[3]])
print(b)


tsess = tf.Session()
result1, result2 = tsess.run([a,b])
print(result1)
print(" ")
print(result2)

Tensor("Const_20:0", shape=(1, 4, 6, 1), dtype=int32)


TypeError: Expected binary or unicode string, got None